<a href="https://colab.research.google.com/github/boesuwanan/Senior-Research/blob/main/%5BSR%5D_linear_regression_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#data

In [ ]:
import pandas as pd

url = 'https://docs.google.com/spreadsheets/d/1p-1vpl6It4-7kzlonay-lAxi2X2RvepL/export?gid=1844082685&format=csv'
raw_df = pd.read_csv(url)
df = raw_df.drop(["monthly helper","quarterly helper", "quarterly helper", "year helper"], axis='columns')

df.head(3)

,ticks,sector,date,esg_OVERALL,esg_E,esg_S,esg_G,stringency_index,RETURN,VOLATILITY_260D_CALC,...,LIQUIDITY,ln_liq,BS_TOT_ASSET,ln_total_asset,RETURN_ON_ASSET,TOT_DEBT_TO_TOT_EQY,ln_debt_to_e,PX_TO_BOOK_RATIO,ln_PTB,SALES_GROWTH
0,AAV TB Equity,Industrials,1/1/2022,3.7,3.04,4.66,3.34,1282.03,-0.015873,42.116,...,1321.028191,7.186166,67589.4934,11.121208,-10.6392,337.8337,5.822554,1.2225,0.200898,55.8186
1,AAV TB Equity,Industrials,1/2/2022,3.7,3.04,4.66,3.34,1138.96,0.007968,41.226,...,2218.380655,7.704533,67589.4934,11.121208,-10.6392,337.8337,5.822554,1.2323,0.208882,55.8186
2,AAV TB Equity,Industrials,1/3/2022,3.7,3.04,4.66,3.34,1334.86,0.023530,40.338,...,1235.268015,7.119043,67589.4934,11.121208,-10.6392,337.8337,5.822554,2.1518,0.766305,55.8186


In [ ]:
df['year'] = pd.to_datetime(df['date']).dt.year

In [ ]:
df_precovid = df[df['year'] < 2020]
df_covid = df[df['year'] == 2020]
df_postcovid = df[df['year'] > 2020]

#export

## df

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Define your formulas for each model
ols_formulas = [
    'RETURN ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq ',
    'ln_vol ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq',
    'ln_liq ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector',
    #
    'RETURN ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq ',
    'ln_vol ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq',
    'ln_liq ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector',
    #
    'RETURN ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq ',
    'ln_vol ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq',
    'ln_liq ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector',
    #
    'RETURN ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq ',
    'ln_vol ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq',
    'ln_liq ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector'
]

# Define the ESG type for each model (or use a default value)
esg_types = [
    'Overall',  # Define the ESG type for the first model
    'Overall',
    'Overall',
    'Environmental',
    'Environmental',
    'Environmental',
    'Social',
    'Social',
    'Social',
    'Governance',
    'Governance',
    'Governance',
    # Define other ESG types for other models
]

# Dataframe to store coefficients, t-values, p-values, and standard errors
coefficients_df = pd.DataFrame()

# Loop through each formula, fit the model, and extract coefficients, t-values, p-values, and standard errors
for formula, esg_type in zip(ols_formulas, esg_types):
    OLS = ols(formula=formula, data=df)
    model = OLS.fit()
    results_summary = model.summary()
    coefficients = model.params
    tvalues = model.tvalues
    pvalues = model.pvalues
    std_errors = model.bse

    # Determine the dependent variable
    dependent_variable = formula.split('~')[0].strip()

    # Combine coefficients, t-values, p-values, and standard errors into one DataFrame
    model_info_df = pd.DataFrame({
        'Dependent Variable': [dependent_variable] * len(coefficients),
        'ESG Type': [esg_type] * len(coefficients),
        'Coefficient': coefficients,
        'T-value': tvalues,
        'P-value': pvalues,
        'Standard Error': std_errors
    })

    # Add a column for the formula used
    model_info_df['Formula'] = formula

    # Append the information for this model to the overall DataFrame
    coefficients_df = coefficients_df._append(model_info_df)

    # # Reset index to make the current index a column
coefficients_df.reset_index(inplace=True)

# Display the coefficients DataFrame
coefficients_df

,index,Dependent Variable,ESG Type,Coefficient,T-value,P-value,Standard Error,Formula
0,Intercept,RETURN,Overall,-0.147801,-7.441814,1.170436e-13,0.019861,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
1,sector[T.Consumer Discretionary],RETURN,Overall,0.018072,2.530331,1.142722e-02,0.007142,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
2,sector[T.Consumer Staples],RETURN,Overall,0.013031,1.768767,7.699636e-02,0.007367,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
3,sector[T.Energy],RETURN,Overall,0.034755,4.031487,5.628464e-05,0.008621,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
4,sector[T.Financials],RETURN,Overall,0.017442,2.519012,1.180071e-02,0.006924,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
...,...,...,...,...,...,...,...,...
231,ln_total_asset,ln_liq,Governance,-0.113865,-9.943166,4.509079e-23,0.011452,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
232,RETURN_ON_ASSET,ln_liq,Governance,0.015146,6.911527,5.422168e-12,0.002191,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
233,ln_debt_to_e,ln_liq,Governance,0.035386,2.703332,6.889007e-03,0.013090,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
234,ln_PTB,ln_liq,Governance,-0.027516,-1.288765,1.975419e-01,0.021351,ln_liq ~ esg_G + (esg_G*stringency_index) + st...


In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
coeff_df = coefficients_df.pivot_table(index='index',
                                       columns='Formula',
                                       values='Coefficient')

# Flatten the multi-level column index
coeff_df.columns = ['_'.join(map(str, col)).strip() for col in coeff_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
coeff_df.reset_index(inplace=True)
coeff_df

,index,R_E_T_U_R_N_ _~_ _e_s_g___E_ _+_ _(_e_s_g___E_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q_,R_E_T_U_R_N_ _~_ _e_s_g___G_ _+_ _(_e_s_g___G_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q_,R_E_T_U_R_N_ _~_ _e_s_g___O_V_E_R_A_L_L_ _+_ _(_e_s_g___O_V_E_R_A_L_L_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q_,R_E_T_U_R_N_ _~_ _e_s_g___S_ _+_ _(_e_s_g___S_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q_,l_n___l_i_q_ _~_ _e_s_g___E_ _+_ _(_e_s_g___E_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r,l_n___l_i_q_ _~_ _e_s_g___G_ _+_ _(_e_s_g___G_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r,l_n___l_i_q_ _~_ _e_s_g___O_V_E_R_A_L_L_ _+_ _(_e_s_g___O_V_E_R_A_L_L_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r,l_n___l_i_q_ _~_ _e_s_g___S_ _+_ _(_e_s_g___S_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r,l_n___v_o_l_ _~_ _e_s_g___E_ _+_ _(_e_s_g___E_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q,l_n___v_o_l_ _~_ _e_s_g___G_ _+_ _(_e_s_g___G_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q,l_n___v_o_l_ _~_ _e_s_g___O_V_E_R_A_L_L_ _+_ _(_e_s_g___O_V_E_R_A_L_L_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q,l_n___v_o_l_ _~_ _e_s_g___S_ _+_ _(_e_s_g___S_*_s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_)_ _+_ _s_t_r_i_n_g_e_n_c_y___i_n_d_e_x_ _+_ _l_n___t_o_t_a_l___a_s_s_e_t_ _+_ _R_E_T_U_R_N___O_N___A_S_S_E_T_ _+_ _l_n___d_e_b_t___t_o___e_ _+_ _l_n___P_T_B_ _+_ _S_A_L_E_S___G_R_O_W_T_H_ _+_ _s_e_c_t_o_r_ _+_ _l_n___l_i_q
0,Intercept,-1.488175e-01,-1.273240e-01,-1.478011e-01,-1.512817e-01,8.759612,9.242104,8.894385,8.909200,3.196145,3.336236,3.297578,3.249775
1,RETURN_ON_ASSET,-7.602398e-04,-7.535723e-04,-7.391826e-04,-7.784343e-04,0.015616,0.015146,0.015253,0.014629,-0.000471,-0.000751,-0.000336,-0.000860
2,SALES_GROWTH,1.742816e-05,1.613389e-05,1.683414e-

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
sd_df = coefficients_df.pivot_table(index='index',
                                       columns='Formula',
                                       values='Standard Error')

# Flatten the multi-level column index
sd_df.columns = ['_'.join(map(str, col)).strip() for col in sd_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
sd_df.reset_index(inplace=True)

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
p_df = coefficients_df.pivot_table(index='index',
                                       columns='Formula',
                                       values='P-value')

# Flatten the multi-level column index
p_df.columns = ['_'.join(map(str, col)).strip() for col in p_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
p_df.reset_index(inplace=True)

In [ ]:
# Assuming coefficients_df is your DataFrame
p_df.to_csv('p_df.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

# from google.colab import files
# files.download('p_df.csv')


## df_precovid

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Define your formulas for each model
ols_formulas = [
    'RETURN ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks ',
    'ln_vol ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks'
]

# Define the ESG type for each model (or use a default value)
esg_types = [
    'Overall',  # Define the ESG type for the first model
    'Overall',
    'Overall',
    'Environmental',
    'Environmental',
    'Environmental',
    'Social',
    'Social',
    'Social',
    'Governance',
    'Governance',
    'Governance',
    # Define other ESG types for other models
]

# Dataframe to store coefficients, t-values, p-values, and standard errors
coefficients_df_precovid = pd.DataFrame()

# Loop through each formula, fit the model, and extract coefficients, t-values, p-values, and standard errors
for formula, esg_type in zip(ols_formulas, esg_types):
    OLS = ols(formula=formula, data=df_precovid)
    model = OLS.fit()
    results_summary = model.summary()
    coefficients = model.params
    tvalues = model.tvalues
    pvalues = model.pvalues
    std_errors = model.bse

    # Determine the dependent variable
    dependent_variable = formula.split('~')[0].strip()

    # Combine coefficients, t-values, p-values, and standard errors into one DataFrame
    model_info_df = pd.DataFrame({
        'Dependent Variable': [dependent_variable] * len(coefficients),
        'ESG Type': [esg_type] * len(coefficients),
        'Coefficient': coefficients,
        'T-value': tvalues,
        'P-value': pvalues,
        'Standard Error': std_errors
    })

    # Add a column for the formula used
    model_info_df['Formula'] = formula

    # Append the information for this model to the overall DataFrame
    coefficients_df_precovid = coefficients_df_precovid._append(model_info_df)

    # # Reset index to make the current index a column
coefficients_df_precovid.reset_index(inplace=True)

# Display the coefficients DataFrame
# coefficients_df_precovid

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
coeff_df = coefficients_df_precovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Coefficient')

# Flatten the multi-level column index
coeff_df.columns = ['_'.join(map(str, col)).strip() for col in coeff_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
coeff_df.reset_index(inplace=True)

# Assuming coefficients_df is your DataFrame
coeff_df.to_csv('coeff_df_precovid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('coeff_df_precovid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
sd_df = coefficients_df_precovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Standard Error')

# Flatten the multi-level column index
sd_df.columns = ['_'.join(map(str, col)).strip() for col in sd_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
sd_df.reset_index(inplace=True)

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
p_df = coefficients_df_precovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='P-value')

# Flatten the multi-level column index
p_df.columns = ['_'.join(map(str, col)).strip() for col in p_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
p_df.reset_index(inplace=True)

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
t_df = coefficients_df_precovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='T-value')

# Flatten the multi-level column index
t_df.columns = ['_'.join(map(str, col)).strip() for col in t_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
t_df.reset_index(inplace=True)

t_df.to_csv('t_df_precovid.csv', index=False)
from google.colab import files
files.download('t_df_precovid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Assuming coefficients_df is your DataFrame
p_df.to_csv('p_df_precovid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('p_df_precovid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## df_covid

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Define your formulas for each model
ols_formulas = [
    'RETURN ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks'
]

# Define the ESG type for each model (or use a default value)
esg_types = [
    'Overall',  # Define the ESG type for the first model
    'Overall',
    'Overall',
    'Environmental',
    'Environmental',
    'Environmental',
    'Social',
    'Social',
    'Social',
    'Governance',
    'Governance',
    'Governance',
    # Define other ESG types for other models
]

# Dataframe to store coefficients, t-values, p-values, and standard errors
coefficients_df_covid = pd.DataFrame()

# Loop through each formula, fit the model, and extract coefficients, t-values, p-values, and standard errors
for formula, esg_type in zip(ols_formulas, esg_types):
    OLS = ols(formula=formula, data=df_covid)
    model = OLS.fit()
    results_summary = model.summary()
    coefficients = model.params
    tvalues = model.tvalues
    pvalues = model.pvalues
    std_errors = model.bse

    # Determine the dependent variable
    dependent_variable = formula.split('~')[0].strip()

    # Combine coefficients, t-values, p-values, and standard errors into one DataFrame
    model_info_df = pd.DataFrame({
        'Dependent Variable': [dependent_variable] * len(coefficients),
        'ESG Type': [esg_type] * len(coefficients),
        'Coefficient': coefficients,
        'T-value': tvalues,
        'P-value': pvalues,
        'Standard Error': std_errors
    })

    # Add a column for the formula used
    model_info_df['Formula'] = formula

    # Append the information for this model to the overall DataFrame
    coefficients_df_covid = coefficients_df_covid._append(model_info_df)

    # # Reset index to make the current index a column
coefficients_df_covid.reset_index(inplace=True)

# Display the coefficients DataFrame
coefficients_df_covid

,index,Dependent Variable,ESG Type,Coefficient,T-value,P-value,Standard Error,Formula
0,Intercept,RETURN,Overall,-2.731002,-2.832649,4.769648e-03,0.964116,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
1,sector[T.Consumer Discretionary],RETURN,Overall,0.503933,3.840509,1.356806e-04,0.131215,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
2,sector[T.Consumer Staples],RETURN,Overall,0.366408,6.846485,1.857387e-11,0.053518,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
3,sector[T.Energy],RETURN,Overall,1.135476,4.930597,1.059421e-06,0.230292,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
4,sector[T.Financials],RETURN,Overall,0.419494,3.301506,1.018102e-03,0.127062,RETURN ~ esg_OVERALL + (esg_OVERALL*stringency...
...,...,...,...,...,...,...,...,...
903,ln_total_asset,ln_liq,Governance,-0.760028,-2.095038,3.658075e-02,0.362775,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
904,RETURN_ON_ASSET,ln_liq,Governance,0.023128,1.833592,6.720326e-02,0.012613,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
905,ln_debt_to_e,ln_liq,Governance,-0.150923,-1.643618,1.007721e-01,0.091823,ln_liq ~ esg_G + (esg_G*stringency_index) + st...
906,ln_PTB,ln_liq,Governance,0.481195,4.622434,4.634248e-06,0.104100,ln_liq ~ esg_G + (esg_G*stringency_index) + st...


In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
coeff_df = coefficients_df_covid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Coefficient')

# Flatten the multi-level column index
coeff_df.columns = ['_'.join(map(str, col)).strip() for col in coeff_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
coeff_df.reset_index(inplace=True)

# Assuming coefficients_df is your DataFrame
coeff_df.to_csv('coeff_df_covid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('coeff_df_covid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
sd_df = coefficients_df_covid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Standard Error')

# Flatten the multi-level column index
sd_df.columns = ['_'.join(map(str, col)).strip() for col in sd_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
sd_df.reset_index(inplace=True)

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
p_df = coefficients_df_covid.pivot_table(index='index',
                                       columns='Formula',
                                       values='P-value')

# Flatten the multi-level column index
p_df.columns = ['_'.join(map(str, col)).strip() for col in p_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
p_df.reset_index(inplace=True)
# Assuming coefficients_df is your DataFrame
p_df.to_csv('p_df_covid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('p_df_covid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
t_df = coefficients_df_covid.pivot_table(index='index',
                                       columns='Formula',
                                       values='T-value')

# Flatten the multi-level column index
t_df.columns = ['_'.join(map(str, col)).strip() for col in t_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
t_df.reset_index(inplace=True)
# Assuming coefficients_df is your DataFrame
t_df.to_csv('t_df_covid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('t_df_covid.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Assuming coefficients_df is your DataFrame
p_df.to_csv('p_df_covid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('p_df_covid.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## df_postcovid

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Define your formulas for each model
ols_formulas = [
    'RETURN ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks',
    #
    'RETURN ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_vol ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks',
    'ln_liq ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks'
]

# Define the ESG type for each model (or use a default value)
esg_types = [
    'Overall',  # Define the ESG type for the first model
    'Overall',
    'Overall',
    'Environmental',
    'Environmental',
    'Environmental',
    'Social',
    'Social',
    'Social',
    'Governance',
    'Governance',
    'Governance',
    # Define other ESG types for other models
]

# Dataframe to store coefficients, t-values, p-values, and standard errors
coefficients_df_postcovid = pd.DataFrame()

# Loop through each formula, fit the model, and extract coefficients, t-values, p-values, and standard errors
for formula, esg_type in zip(ols_formulas, esg_types):
    OLS = ols(formula=formula, data=df_postcovid)
    model = OLS.fit()
    results_summary = model.summary()
    coefficients = model.params
    tvalues = model.tvalues
    pvalues = model.pvalues
    std_errors = model.bse

    # Determine the dependent variable
    dependent_variable = formula.split('~')[0].strip()

    # Combine coefficients, t-values, p-values, and standard errors into one DataFrame
    model_info_df = pd.DataFrame({
        'Dependent Variable': [dependent_variable] * len(coefficients),
        'ESG Type': [esg_type] * len(coefficients),
        'Coefficient': coefficients,
        'T-value': tvalues,
        'P-value': pvalues,
        'Standard Error': std_errors
    })

    # Add a column for the formula used
    model_info_df['Formula'] = formula

    # Append the information for this model to the overall DataFrame
    coefficients_df_postcovid = coefficients_df_postcovid._append(model_info_df)

    # # Reset index to make the current index a column
coefficients_df_postcovid.reset_index(inplace=True)

# Display the coefficients DataFrame
coefficients_df_postcovid

,index,Dependent Variable,ESG Type,Coefficient,T-value,P-value,Standard Error,Formula
0,Intercept,RETURN,Overall,-0.823801,-3.163454,1.588930e-03,0.260412,RETURN ~ esg_OVERALL + ln_total_asset + RETURN...
1,sector[T.Consumer Discretionary],RETURN,Overall,-0.069061,-4.706154,2.746144e-06,0.014675,RETURN ~ esg_OVERALL + ln_total_asset + RETURN...
2,sector[T.Consumer Staples],RETURN,Overall,-0.058864,-2.627886,8.675410e-03,0.022400,RETURN ~ esg_OVERALL + ln_total_asset + RETURN...
3,sector[T.Energy],RETURN,Overall,0.032100,0.743293,4.574152e-01,0.043186,RETURN ~ esg_OVERALL + ln_total_asset + RETURN...
4,sector[T.Financials],RETURN,Overall,-0.057246,-0.826689,4.085388e-01,0.069247,RETURN ~ esg_OVERALL + ln_total_asset + RETURN...
...,...,...,...,...,...,...,...,...
1443,ln_total_asset,ln_liq,Governance,-0.349572,-2.103162,3.561017e-02,0.166213,ln_liq ~ esg_G + ln_total_asset + RETURN_ON_AS...
1444,RETURN_ON_ASSET,ln_liq,Governance,0.004213,1.342757,1.795445e-01,0.003138,ln_liq ~ esg_G + ln_total_asset + RETURN_ON_AS...
1445,ln_debt_to_e,ln_liq,Governance,-0.012979,-0.334945,7.377114e-01,0.038751,ln_liq ~ esg_G + ln_total_asset + RETURN_ON_AS...
1446,ln_PTB,ln_liq,Governance,1.049725,11.495973,1.998485e-29,0.091312,ln_liq ~ esg_G + ln_total_asset + RETURN_ON_AS...


In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
coeff_df = coefficients_df_postcovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Coefficient')

# Flatten the multi-level column index
coeff_df.columns = ['_'.join(map(str, col)).strip() for col in coeff_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
coeff_df.reset_index(inplace=True)

# Assuming coefficients_df is your DataFrame
coeff_df.to_csv('coeff_df_postcovid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('coeff_df_postcovid.csv')

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
sd_df = coefficients_df_postcovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='Standard Error')

# Flatten the multi-level column index
sd_df.columns = ['_'.join(map(str, col)).strip() for col in sd_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
sd_df.reset_index(inplace=True)

In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
p_df = coefficients_df_postcovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='P-value')

# Flatten the multi-level column index
p_df.columns = ['_'.join(map(str, col)).strip() for col in p_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
p_df.reset_index(inplace=True)

# Assuming coefficients_df is your DataFrame
p_df.to_csv('p_df_postcovid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('p_df_postcovid.csv')


In [ ]:
# Pivot the DataFrame to have models as columns and variables as indices
t_df = coefficients_df_postcovid.pivot_table(index='index',
                                       columns='Formula',
                                       values='T-value')

# Flatten the multi-level column index
t_df.columns = ['_'.join(map(str, col)).strip() for col in t_df.columns.values]

# Reset index to make ESG Type, Dependent Variable, and Formula as columns
t_df.reset_index(inplace=True)

In [ ]:
# Assuming coefficients_df is your DataFrame
t_df.to_csv('t_df_postcovid.csv', index=False)

# After saving the file, you can download it using various methods depending on your environment.
# If you are working in a Jupyter Notebook, you can use the following:

from google.colab import files
files.download('t_df_postcovid.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# overall

## return

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     4.128
Date:                Sat, 20 Apr 2024   Prob (F-statistic):           9.66e-42
Time:                        07:56:21   Log-Likelihood:                 4853.1
No. Observations:                4818   AIC:                            -9478.
Df Residuals:                    4704   BIC:                            -8739.
Df Model:                         113                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.1564      0.058     -2.705      0.007      -0.270      -0.043
sector[T.Consumer Discretionary]    -0.0134      0.007     -2.005      0.045      -0.026      -0.000
sector[T.Consumer Staples]          -0.0117      0.008     -1.428      0.153      -0.028       0.004
sector[T.Energy]                     0.0897      0.013      6.962      0.000       0.064       0.115
sector[T.Financials]                 0.0536      0.016      3.304      0.001       0.022       0.085
sector[T.Health Care]               -0.0210      0.008     -2.472      0.013      -0.038      -0.004
sector[T.Industrials]                0.0046      0.031      0.150      0.881      -0.055       0.065
sector[T.Information Technology]    -0.0200      0.009     -2.181      0.029      -0.038      -0.002
sector[T.Materials]                  0.0502      0.009      5.721      0.000       0.033       0.067
sector[T.Real Estate]                0.0813      0.009      9.119      0.000       0.064       0.099
sector[T.Utilities]                  0.0133      0.009      1.432      0.152      -0.005       0.032
ticks[T.ACE TB Equity]              -0.0161      0.028     -0.567      0.571      -0.072       0.040
ticks[T.ADVANC TB Equity]           -0.0688      0.019     -3.535      0.000      -0.107      -0.031
ticks[T.AMATA TB Equity]            -0.0211      0.025     -0.857      0.392      -0.069       0.027
ticks[T.AOT TB Equity]              -0.0365      0.034     -1.068      0.286      -0.103       0.030
ticks[T.AP TB Equity]                0.0136      0.011      1.264      0.206      -0.008       0.035
ticks[T.AWC TB Equity]               0.0420      0.018      2.295      0.022       0.006       0.078
ticks[T.BAFS TB Equity]              0.0227      0.041      0.555      0.579      -0.057       0.103
ticks[T.BAM TB Equity]              -0.0346      0.025     -1.369      0.171      -0.084       0.015
ticks[T.BANPU TB Equity]             0.0074      0.013      0.564      0.573      -0.018       0.033
ticks[T.BAY TB Equity]               0.1007      0.017      5.761      0.000       0.066       0.135
ticks[T.BBL TB Equity]               0.0645      0.018      3.549      0.000       0.029       0.100
ticks[T.BCH TB Equity]              -0.0473      0.012     -3.841      0.000      -0.071      -0.023
ticks[T.BCP TB Equity]               0.0357      0.012      2.925      0.003       0.012       0.060
ticks[T.BDMS TB Equity]             -0.0115      0.015     -0.765      0.445      -0.041       0.018
ticks[T.BEAUTY TB Equity]           -0.1377      0.021     -6.457      0.000      -0.179      -0.096
ticks[T.BEC TB Equity]              -0.0584      0.011     -5.332      0.000      -0.080      -0.037
ticks[T.BEM TB Equity]              -0.0142      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     4.097
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           7.49e-23
Time:                        09:59:14   Log-Likelihood:                 2880.1
No. Observations:                2464   AIC:                            -5638.
Df Residuals:                    2403   BIC:                            -5284.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.2129      0.115     -1.856      0.064      -0.438       0.012
sector[T.Consumer Discretionary]     0.0668      0.021      3.134      0.002       0.025       0.109
sector[T.Consumer Staples]           0.0713      0.015      4.851      0.000       0.042       0.100
sector[T.Energy]                     0.1881      0.020      9.542      0.000       0.149       0.227
sector[T.Financials]                 0.1212      0.019      6.361      0.000       0.084       0.159
sector[T.Health Care]                0.0032      0.018      0.180      0.858      -0.032       0.038
sector[T.Industrials]                0.0828      0.015      5.471      0.000       0.053       0.113
sector[T.Information Technology]     0.0562      0.022      2.582      0.010       0.014       0.099
sector[T.Materials]                  0.1068      0.015      7.047      0.000       0.077       0.137
sector[T.Real Estate]                0.1347      0.020      6.601      0.000       0.095       0.175
sector[T.Utilities]                  0.1053      0.016      6.699      0.000       0.074       0.136
ticks[T.AOT TB Equity]               0.0033      0.016      0.211      0.833      -0.028       0.034
ticks[T.AP TB Equity]                0.0611      0.012      4.940      0.000       0.037       0.085
ticks[T.BANPU TB Equity]             0.0158      0.013      1.230      0.219      -0.009       0.041
ticks[T.BAY TB Equity]               0.0530      0.021      2.560      0.011       0.012       0.094
ticks[T.BBL TB Equity]               0.0516      0.022      2.308      0.021       0.008       0.096
ticks[T.BCH TB Equity]              -0.0027      0.018     -0.154      0.877      -0.037       0.032
ticks[T.BCP TB Equity]               0.0488      0.014      3.451      0.001       0.021       0.077
ticks[T.BDMS TB Equity]              0.0209      0.013      1.641      0.101      -0.004       0.046
ticks[T.BEAUTY TB Equity]           -0.1266      0.036     -3.516      0.000      -0.197      -0.056
ticks[T.BEC TB Equity]               0.0084      0.032      0.267      0.790      -0.053       0.070
ticks[T.BEM TB Equity]               0.0016      0.012      0.131      0.896      -0.023       0.026
ticks[T.BGRIM TB Equity]            -0.0308      0.015     -2.026      0.043      -0.061      -0.001
ticks[T.BH TB Equity]               -0.0150      0.013     -1.150      0.250      -0.041       0.011
ticks[T.BJC TB Equity]               0.0277      0.014      1.992      0.047       0.000       0.055
ticks[T.BTS TB Equity]               0.0631      0.014      4.447      0.000       0.035       0.091
ticks[T.CPALL TB Equity]            -0.0833      0.014     -5.829      0.000      -0.111      -0.055
ticks[T.CPF TB Equity]               0.0921      0.01

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     5.503
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           9.99e-31
Time:                        10:00:20   Log-Likelihood:                 455.56
No. Observations:                 672   AIC:                            -781.1
Df Residuals:                     607   BIC:                            -488.0
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -2.7310      0.964     -2.833      0.005      -4.624      -0.838
sector[T.Consumer Discretionary]     0.5039      0.131      3.841      0.000       0.246       0.762
sector[T.Consumer Staples]           0.3664      0.054      6.846      0.000       0.261       0.472
sector[T.Energy]                     1.1355      0.230      4.931      0.000       0.683       1.588
sector[T.Financials]                 0.4195      0.127      3.302      0.001       0.170       0.669
sector[T.Health Care]                0.2547      0.092      2.777      0.006       0.075       0.435
sector[T.Industrials]                0.5534      0.158      3.502      0.000       0.243       0.864
sector[T.Information Technology]     0.4302      0.096      4.474      0.000       0.241       0.619
sector[T.Materials]                  0.6627      0.116      5.702      0.000       0.434       0.891
sector[T.Real Estate]                0.7301      0.087      8.434      0.000       0.560       0.900
sector[T.Utilities]                  0.4944      0.123      4.028      0.000       0.253       0.735
ticks[T.AOT TB Equity]               0.0269      0.092      0.291      0.771      -0.154       0.208
ticks[T.AP TB Equity]                0.1512      0.044      3.456      0.001       0.065       0.237
ticks[T.AWC TB Equity]              -0.1435      0.162     -0.885      0.376      -0.462       0.175
ticks[T.BANPU TB Equity]             0.0088      0.087      0.101      0.920      -0.162       0.179
ticks[T.BAY TB Equity]               0.1174      0.163      0.720      0.472      -0.203       0.438
ticks[T.BBL TB Equity]              -0.0462      0.195     -0.237      0.812      -0.429       0.336
ticks[T.BCH TB Equity]              -0.1077      0.044     -2.465      0.014      -0.194      -0.022
ticks[T.BCP TB Equity]               0.3370      0.106      3.182      0.002       0.129       0.545
ticks[T.BDMS TB Equity]              0.2721      0.070      3.885      0.000       0.135       0.410
ticks[T.BEAUTY TB Equity]            0.2503      0.325      0.770      0.442      -0.388       0.889
ticks[T.BEC TB Equity]               0.6416      0.174      3.696      0.000       0.301       0.982
ticks[T.BEM TB Equity]               0.0888      0.069      1.294      0.196      -0.046       0.224
ticks[T.BGRIM TB Equity]            -0.4330      0.088     -4.927      0.000      -0.606      -0.260
ticks[T.BH TB Equity]                0.0903      0.054      1.665      0.097      -0.016       0.197
ticks[T.BJC TB Equity]              -0.1774      0.199     -0.890      0.374      -0.569       0.214
ticks[T.BTS TB Equity]               0.4721      0.154      3.060      0.002       0.169       0.775
ticks[T.CPALL TB Equity]            -0.2699      0.04

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     2.790
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           4.10e-18
Time:                        10:04:24   Log-Likelihood:                 2081.0
No. Observations:                1682   AIC:                            -3940.
Df Residuals:                    1571   BIC:                            -3338.
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.8238      0.260     -3.163      0.002      -1.335      -0.313
sector[T.Consumer Discretionary]    -0.0691      0.015     -4.706      0.000      -0.098      -0.040
sector[T.Consumer Staples]          -0.0589      0.022     -2.628      0.009      -0.103      -0.015
sector[T.Energy]                     0.0321      0.043      0.743      0.457      -0.053       0.117
sector[T.Financials]                -0.0572      0.069     -0.827      0.409      -0.193       0.079
sector[T.Health Care]               -0.0411      0.013     -3.153      0.002      -0.067      -0.016
sector[T.Industrials]               -0.0633      0.040     -1.597      0.111      -0.141       0.014
sector[T.Information Technology]    -0.0671      0.016     -4.294      0.000      -0.098      -0.036
sector[T.Materials]                  0.0204      0.024      0.849      0.396      -0.027       0.068
sector[T.Real Estate]                0.0849      0.028      3.071      0.002       0.031       0.139
sector[T.Utilities]                 -0.0743      0.036     -2.075      0.038      -0.145      -0.004
ticks[T.ACE TB Equity]               0.0881      0.046      1.917      0.055      -0.002       0.178
ticks[T.ADVANC TB Equity]           -0.2892      0.076     -3.808      0.000      -0.438      -0.140
ticks[T.AMATA TB Equity]            -0.0310      0.022     -1.403      0.161      -0.074       0.012
ticks[T.AOT TB Equity]              -0.2356      0.048     -4.885      0.000      -0.330      -0.141
ticks[T.AP TB Equity]                0.0205      0.017      1.195      0.232      -0.013       0.054
ticks[T.AWC TB Equity]               0.0120      0.043      0.281      0.779      -0.071       0.095
ticks[T.BAFS TB Equity]              0.0495      0.049      1.003      0.316      -0.047       0.146
ticks[T.BAM TB Equity]               0.0079      0.031      0.250      0.802      -0.054       0.070
ticks[T.BANPU TB Equity]            -0.0411      0.033     -1.243      0.214      -0.106       0.024
ticks[T.BAY TB Equity]               0.0290      0.054      0.532      0.595      -0.078       0.136
ticks[T.BBL TB Equity]              -0.0127      0.069     -0.183      0.855      -0.149       0.123
ticks[T.BCH TB Equity]              -0.0208      0.020     -1.045      0.296      -0.060       0.018
ticks[T.BCP TB Equity]               0.0493      0.021      2.325      0.020       0.008       0.091
ticks[T.BDMS TB Equity]             -0.1233      0.048     -2.583      0.010      -0.217      -0.030
ticks[T.BEAUTY TB Equity]            0.0620      0.091      0.685      0.494      -0.116       0.240
ticks[T.BEC TB Equity]              -0.0457      0.025     -1.847      0.065      -0.094       0.003
ticks[T.BEM TB Equity]              -0.0617      0.03

## vol

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     102.5
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:10:24   Log-Likelihood:                 935.03
No. Observations:                2464   AIC:                            -1748.
Df Residuals:                    2403   BIC:                            -1394.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.3744      0.253     13.360      0.000       2.879       3.870
sector[T.Consumer Discretionary]     0.2301      0.047      4.900      0.000       0.138       0.322
sector[T.Consumer Staples]          -0.0166      0.032     -0.512      0.609      -0.080       0.047
sector[T.Energy]                     0.2008      0.043      4.626      0.000       0.116       0.286
sector[T.Financials]                 0.0063      0.042      0.151      0.880      -0.076       0.089
sector[T.Health Care]               -0.1202      0.039     -3.060      0.002      -0.197      -0.043
sector[T.Industrials]               -0.0055      0.033     -0.166      0.868      -0.071       0.060
sector[T.Information Technology]     0.1306      0.048      2.725      0.006       0.037       0.224
sector[T.Materials]                  0.1744      0.033      5.225      0.000       0.109       0.240
sector[T.Real Estate]               -0.1249      0.045     -2.780      0.005      -0.213      -0.037
sector[T.Utilities]                 -0.0032      0.035     -0.092      0.926      -0.071       0.065
ticks[T.AOT TB Equity]               0.1925      0.035      5.522      0.000       0.124       0.261
ticks[T.AP TB Equity]               -0.0262      0.027     -0.963      0.336      -0.080       0.027
ticks[T.BANPU TB Equity]            -0.0641      0.028     -2.272      0.023      -0.119      -0.009
ticks[T.BAY TB Equity]               0.3265      0.046      7.164      0.000       0.237       0.416
ticks[T.BBL TB Equity]              -0.1992      0.049     -4.044      0.000      -0.296      -0.103
ticks[T.BCH TB Equity]              -0.1077      0.039     -2.788      0.005      -0.183      -0.032
ticks[T.BCP TB Equity]              -0.2015      0.031     -6.469      0.000      -0.263      -0.140
ticks[T.BDMS TB Equity]             -0.0554      0.028     -1.976      0.048      -0.110      -0.000
ticks[T.BEAUTY TB Equity]            0.2693      0.079      3.397      0.001       0.114       0.425
ticks[T.BEC TB Equity]               0.1835      0.070      2.639      0.008       0.047       0.320
ticks[T.BEM TB Equity]              -0.1901      0.027     -6.946      0.000      -0.244      -0.136
ticks[T.BGRIM TB Equity]             0.0789      0.033      2.358      0.018       0.013       0.145
ticks[T.BH TB Equity]                0.0429      0.029      1.495      0.135      -0.013       0.099
ticks[T.BJC TB Equity]               0.0848      0.031      2.769      0.006       0.025       0.145
ticks[T.BTS TB Equity]              -0.2734      0.031     -8.742      0.000      -0.335      -0.212
ticks[T.CPALL TB Equity]            -0.2030      0.031     -6.453      0.000      -0.265      -0.141
ticks[T.CPF TB Equity]               0.0612      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     67.91
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          1.83e-235
Time:                        10:10:59   Log-Likelihood:                 541.63
No. Observations:                 672   AIC:                            -953.3
Df Residuals:                     607   BIC:                            -660.1
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -2.3732      0.848     -2.798      0.005      -4.039      -0.707
sector[T.Consumer Discretionary]     1.1532      0.115      9.990      0.000       0.927       1.380
sector[T.Consumer Staples]           0.1266      0.047      2.688      0.007       0.034       0.219
sector[T.Energy]                     1.1001      0.203      5.430      0.000       0.702       1.498
sector[T.Financials]                -0.5790      0.112     -5.180      0.000      -0.799      -0.359
sector[T.Health Care]                0.6147      0.081      7.615      0.000       0.456       0.773
sector[T.Industrials]                0.8495      0.139      6.112      0.000       0.577       1.123
sector[T.Information Technology]     0.6897      0.085      8.155      0.000       0.524       0.856
sector[T.Materials]                  0.5930      0.102      5.800      0.000       0.392       0.794
sector[T.Real Estate]                0.3515      0.076      4.616      0.000       0.202       0.501
sector[T.Utilities]                  0.7879      0.108      7.297      0.000       0.576       1.000
ticks[T.AOT TB Equity]               0.3131      0.081      3.855      0.000       0.154       0.473
ticks[T.AP TB Equity]               -0.0149      0.038     -0.388      0.698      -0.090       0.061
ticks[T.AWC TB Equity]              -0.7878      0.143     -5.524      0.000      -1.068      -0.508
ticks[T.BANPU TB Equity]            -0.2698      0.076     -3.527      0.000      -0.420      -0.120
ticks[T.BAY TB Equity]              -1.1990      0.143     -8.358      0.000      -1.481      -0.917
ticks[T.BBL TB Equity]              -1.3841      0.171     -8.073      0.000      -1.721      -1.047
ticks[T.BCH TB Equity]               0.2915      0.038      7.582      0.000       0.216       0.367
ticks[T.BCP TB Equity]               0.4056      0.093      4.354      0.000       0.223       0.589
ticks[T.BDMS TB Equity]              0.0360      0.062      0.584      0.559      -0.085       0.157
ticks[T.BEAUTY TB Equity]            2.1356      0.286      7.464      0.000       1.574       2.697
ticks[T.BEC TB Equity]               1.3620      0.153      8.920      0.000       1.062       1.662
ticks[T.BEM TB Equity]               0.2776      0.060      4.597      0.000       0.159       0.396
ticks[T.BGRIM TB Equity]            -0.0972      0.077     -1.257      0.209      -0.249       0.055
ticks[T.BH TB Equity]                0.2872      0.048      6.016      0.000       0.193       0.381
ticks[T.BJC TB Equity]              -0.9685      0.175     -5.525      0.000      -1.313      -0.624
ticks[T.BTS TB Equity]               0.6704      0.136      4.940      0.000       0.404       0.937
ticks[T.CPALL TB Equity]            -0.3600      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     35.32
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:11:22   Log-Likelihood:                 462.49
No. Observations:                1682   AIC:                            -703.0
Df Residuals:                    1571   BIC:                            -100.5
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.7104      0.682     15.712      0.000       9.373      12.047
sector[T.Consumer Discretionary]     0.5967      0.038     15.535      0.000       0.521       0.672
sector[T.Consumer Staples]           0.6209      0.059     10.590      0.000       0.506       0.736
sector[T.Energy]                     1.7528      0.113     15.506      0.000       1.531       1.975
sector[T.Financials]                 2.2062      0.181     12.172      0.000       1.851       2.562
sector[T.Health Care]                0.0750      0.034      2.197      0.028       0.008       0.142
sector[T.Industrials]                1.4175      0.104     13.658      0.000       1.214       1.621
sector[T.Information Technology]     0.3693      0.041      9.022      0.000       0.289       0.450
sector[T.Materials]                  1.0534      0.063     16.716      0.000       0.930       1.177
sector[T.Real Estate]                0.7373      0.072     10.193      0.000       0.595       0.879
sector[T.Utilities]                  1.2963      0.094     13.831      0.000       1.112       1.480
ticks[T.ACE TB Equity]              -1.4037      0.120    -11.670      0.000      -1.640      -1.168
ticks[T.ADVANC TB Equity]            2.0090      0.199     10.108      0.000       1.619       2.399
ticks[T.AMATA TB Equity]            -0.2658      0.058     -4.594      0.000      -0.379      -0.152
ticks[T.AOT TB Equity]               0.5816      0.126      4.606      0.000       0.334       0.829
ticks[T.AP TB Equity]               -0.1980      0.045     -4.401      0.000      -0.286      -0.110
ticks[T.AWC TB Equity]               1.0896      0.111      9.793      0.000       0.871       1.308
ticks[T.BAFS TB Equity]             -0.9659      0.129     -7.472      0.000      -1.219      -0.712
ticks[T.BAM TB Equity]              -1.1542      0.082    -14.017      0.000      -1.316      -0.993
ticks[T.BANPU TB Equity]             0.8712      0.087     10.066      0.000       0.701       1.041
ticks[T.BAY TB Equity]               2.2663      0.142     15.905      0.000       1.987       2.546
ticks[T.BBL TB Equity]               1.8758      0.181     10.342      0.000       1.520       2.232
ticks[T.BCH TB Equity]              -0.4233      0.052     -8.113      0.000      -0.526      -0.321
ticks[T.BCP TB Equity]               0.5995      0.056     10.799      0.000       0.491       0.708
ticks[T.BDMS TB Equity]              1.3797      0.125     11.038      0.000       1.135       1.625
ticks[T.BEAUTY TB Equity]           -2.9510      0.237    -12.455      0.000      -3.416      -2.486
ticks[T.BEC TB Equity]              -0.5762      0.065     -8.897      0.000      -0.703      -0.449
ticks[T.BEM TB Equity]              -0.0845      0.09

## liq

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     85.14
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:35:29   Log-Likelihood:                -1754.8
No. Observations:                2464   AIC:                             3630.
Df Residuals:                    2404   BIC:                             3978.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.8944      0.748      5.206      0.000       2.427       5.361
sector[T.Consumer Discretionary]     1.3043      0.137      9.496      0.000       1.035       1.574
sector[T.Consumer Staples]           0.4972      0.096      5.188      0.000       0.309       0.685
sector[T.Energy]                     0.9860      0.128      7.720      0.000       0.736       1.237
sector[T.Financials]                 0.2659      0.125      2.130      0.033       0.021       0.511
sector[T.Health Care]                0.5825      0.116      5.006      0.000       0.354       0.811
sector[T.Industrials]                0.6901      0.098      7.020      0.000       0.497       0.883
sector[T.Information Technology]     1.0252      0.141      7.264      0.000       0.748       1.302
sector[T.Materials]                  0.3558      0.099      3.587      0.000       0.161       0.550
sector[T.Real Estate]                1.1209      0.132      8.497      0.000       0.862       1.380
sector[T.Utilities]                  0.5677      0.102      5.541      0.000       0.367       0.769
ticks[T.AOT TB Equity]              -0.7934      0.103     -7.734      0.000      -0.995      -0.592
ticks[T.AP TB Equity]                0.4181      0.081      5.185      0.000       0.260       0.576
ticks[T.BANPU TB Equity]             1.1405      0.081     14.125      0.000       0.982       1.299
ticks[T.BAY TB Equity]              -3.8385      0.111    -34.604      0.000      -4.056      -3.621
ticks[T.BBL TB Equity]              -0.3609      0.147     -2.463      0.014      -0.648      -0.074
ticks[T.BCH TB Equity]               1.0288      0.113      9.097      0.000       0.807       1.251
ticks[T.BCP TB Equity]               0.1503      0.093      1.621      0.105      -0.032       0.332
ticks[T.BDMS TB Equity]             -0.4001      0.083     -4.816      0.000      -0.563      -0.237
ticks[T.BEAUTY TB Equity]            1.7783      0.233      7.621      0.000       1.321       2.236
ticks[T.BEC TB Equity]               1.3383      0.205      6.520      0.000       0.936       1.741
ticks[T.BEM TB Equity]               0.2162      0.081      2.656      0.008       0.057       0.376
ticks[T.BGRIM TB Equity]             0.5095      0.099      5.139      0.000       0.315       0.704
ticks[T.BH TB Equity]               -0.0461      0.085     -0.540      0.589      -0.214       0.121
ticks[T.BJC TB Equity]              -0.2625      0.091     -2.881      0.004      -0.441      -0.084
ticks[T.BTS TB Equity]              -0.1478      0.093     -1.587      0.113      -0.330       0.035
ticks[T.CPALL TB Equity]            -0.5148      0.093     -5.529      0.000      -0.697      -0.332
ticks[T.CPF TB Equity]               0.3406      0.10

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_OVERALL + (esg_OVERALL*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     31.54
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          1.32e-152
Time:                        10:35:46   Log-Likelihood:                -420.86
No. Observations:                 672   AIC:                             969.7
Df Residuals:                     608   BIC:                             1258.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           13.5648      3.507      3.868      0.000       6.678      20.452
sector[T.Consumer Discretionary]     0.5321      0.483      1.103      0.271      -0.416       1.480
sector[T.Consumer Staples]           0.6924      0.195      3.550      0.000       0.309       1.075
sector[T.Energy]                    -0.4780      0.848     -0.564      0.573      -2.143       1.187
sector[T.Financials]                 2.1473      0.460      4.672      0.000       1.245       3.050
sector[T.Health Care]               -0.2987      0.338     -0.885      0.377      -0.962       0.364
sector[T.Industrials]               -0.6983      0.581     -1.202      0.230      -1.839       0.443
sector[T.Information Technology]     1.0397      0.351      2.959      0.003       0.350       1.730
sector[T.Materials]                 -0.2798      0.428     -0.654      0.513      -1.120       0.560
sector[T.Real Estate]                0.9986      0.316      3.159      0.002       0.378       1.619
sector[T.Utilities]                 -0.1252      0.452     -0.277      0.782      -1.013       0.762
ticks[T.AOT TB Equity]              -1.5911      0.334     -4.768      0.000      -2.246      -0.936
ticks[T.AP TB Equity]                0.9217      0.157      5.885      0.000       0.614       1.229
ticks[T.AWC TB Equity]               0.7756      0.596      1.301      0.194      -0.395       1.946
ticks[T.BANPU TB Equity]             2.0436      0.309      6.610      0.000       1.436       2.651
ticks[T.BAY TB Equity]              -2.1245      0.594     -3.576      0.000      -3.291      -0.958
ticks[T.BBL TB Equity]               2.4117      0.711      3.393      0.001       1.016       3.808
ticks[T.BCH TB Equity]               0.8361      0.157      5.317      0.000       0.527       1.145
ticks[T.BCP TB Equity]              -1.1968      0.387     -3.094      0.002      -1.956      -0.437
ticks[T.BDMS TB Equity]             -0.9771      0.255     -3.835      0.000      -1.478      -0.477
ticks[T.BEAUTY TB Equity]           -1.1617      1.196     -0.971      0.332      -3.511       1.188
ticks[T.BEC TB Equity]               0.9458      0.638      1.483      0.139      -0.307       2.198
ticks[T.BEM TB Equity]              -0.4783      0.252     -1.898      0.058      -0.973       0.017
ticks[T.BGRIM TB Equity]             0.9664      0.321      3.009      0.003       0.336       1.597
ticks[T.BH TB Equity]               -0.1577      0.200     -0.790      0.430      -0.550       0.234
ticks[T.BJC TB Equity]               2.2868      0.728      3.143      0.002       0.858       3.716
ticks[T.BTS TB Equity]              -1.4161      0.565     -2.506      0.012      -2.526      -0.306
ticks[T.CPALL TB Equity]            -0.3950      0.15

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_OVERALL + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     62.06
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:35:57   Log-Likelihood:                -1098.1
No. Observations:                1682   AIC:                             2416.
Df Residuals:                    1572   BIC:                             3013.
Df Model:                         109                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.5628      1.703      6.204      0.000       7.223      13.902
sector[T.Consumer Discretionary]     0.2576      0.097      2.658      0.008       0.068       0.448
sector[T.Consumer Staples]           0.3462      0.148      2.340      0.019       0.056       0.636
sector[T.Energy]                     2.3513      0.280      8.410      0.000       1.803       2.900
sector[T.Financials]                 1.9049      0.456      4.180      0.000       1.011       2.799
sector[T.Health Care]                0.0559      0.086      0.648      0.517      -0.113       0.225
sector[T.Industrials]                0.6039      0.262      2.306      0.021       0.090       1.118
sector[T.Information Technology]     0.6739      0.102      6.603      0.000       0.474       0.874
sector[T.Materials]                  0.3084      0.159      1.938      0.053      -0.004       0.621
sector[T.Real Estate]                1.0718      0.181      5.926      0.000       0.717       1.427
sector[T.Utilities]                  0.9157      0.236      3.883      0.000       0.453       1.378
ticks[T.ACE TB Equity]              -0.8443      0.303     -2.783      0.005      -1.439      -0.249
ticks[T.ADVANC TB Equity]           -0.1219      0.502     -0.243      0.808      -1.108       0.864
ticks[T.AMATA TB Equity]             1.0087      0.144      7.003      0.000       0.726       1.291
ticks[T.AOT TB Equity]              -1.2445      0.318     -3.917      0.000      -1.868      -0.621
ticks[T.AP TB Equity]                1.2224      0.109     11.168      0.000       1.008       1.437
ticks[T.AWC TB Equity]               0.3964      0.281      1.410      0.159      -0.155       0.948
ticks[T.BAFS TB Equity]             -1.5005      0.325     -4.623      0.000      -2.137      -0.864
ticks[T.BAM TB Equity]               0.3513      0.208      1.689      0.091      -0.057       0.759
ticks[T.BANPU TB Equity]             1.9758      0.213      9.273      0.000       1.558       2.394
ticks[T.BAY TB Equity]              -1.6367      0.358     -4.574      0.000      -2.339      -0.935
ticks[T.BBL TB Equity]               2.6622      0.454      5.869      0.000       1.772       3.552
ticks[T.BCH TB Equity]               0.5206      0.131      3.966      0.000       0.263       0.778
ticks[T.BCP TB Equity]               0.3787      0.140      2.705      0.007       0.104       0.653
ticks[T.BDMS TB Equity]              0.0878      0.316      0.278      0.781      -0.532       0.708
ticks[T.BEAUTY TB Equity]           -1.2703      0.598     -2.124      0.034      -2.443      -0.097
ticks[T.BEC TB Equity]               0.2070      0.164      1.265      0.206      -0.114       0.528
ticks[T.BEM TB Equity]               0.0236      0.24

# envi

## return

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     4.022
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           6.01e-40
Time:                        10:18:04   Log-Likelihood:                 4847.5
No. Observations:                4818   AIC:                            -9467.
Df Residuals:                    4704   BIC:                            -8728.
Df Model:                         113                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.1473      0.058     -2.539      0.011      -0.261      -0.034
sector[T.Consumer Discretionary]    -0.0141      0.007     -2.116      0.034      -0.027      -0.001
sector[T.Consumer Staples]          -0.0042      0.008     -0.531      0.596      -0.020       0.011
sector[T.Energy]                     0.0834      0.013      6.537      0.000       0.058       0.108
sector[T.Financials]                 0.0602      0.017      3.633      0.000       0.028       0.093
sector[T.Health Care]               -0.0136      0.008     -1.631      0.103      -0.030       0.003
sector[T.Industrials]               -0.0051      0.031     -0.166      0.868      -0.065       0.055
sector[T.Information Technology]    -0.0088      0.009     -1.019      0.308      -0.026       0.008
sector[T.Materials]                  0.0495      0.009      5.617      0.000       0.032       0.067
sector[T.Real Estate]                0.0831      0.009      9.133      0.000       0.065       0.101
sector[T.Utilities]                  0.0107      0.009      1.150      0.250      -0.008       0.029
ticks[T.ACE TB Equity]              -0.0193      0.028     -0.681      0.496      -0.075       0.036
ticks[T.ADVANC TB Equity]           -0.0697      0.020     -3.573      0.000      -0.108      -0.031
ticks[T.AMATA TB Equity]            -0.0247      0.025     -1.002      0.316      -0.073       0.024
ticks[T.AOT TB Equity]              -0.0302      0.034     -0.884      0.377      -0.097       0.037
ticks[T.AP TB Equity]                0.0164      0.011      1.508      0.132      -0.005       0.038
ticks[T.AWC TB Equity]               0.0551      0.018      3.058      0.002       0.020       0.090
ticks[T.BAFS TB Equity]              0.0114      0.041      0.278      0.781      -0.069       0.092
ticks[T.BAM TB Equity]              -0.0390      0.025     -1.542      0.123      -0.089       0.011
ticks[T.BANPU TB Equity]             0.0145      0.013      1.116      0.265      -0.011       0.040
ticks[T.BAY TB Equity]               0.1093      0.017      6.312      0.000       0.075       0.143
ticks[T.BBL TB Equity]               0.0771      0.018      4.292      0.000       0.042       0.112
ticks[T.BCH TB Equity]              -0.0456      0.012     -3.693      0.000      -0.070      -0.021
ticks[T.BCP TB Equity]               0.0278      0.012      2.294      0.022       0.004       0.052
ticks[T.BDMS TB Equity]             -0.0105      0.015     -0.696      0.486      -0.040       0.019
ticks[T.BEAUTY TB Equity]           -0.1452      0.021     -6.835      0.000      -0.187      -0.104
ticks[T.BEC TB Equity]              -0.0534      0.011     -4.887      0.000      -0.075      -0.032
ticks[T.BEM TB Equity]              -0.0035      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     3.916
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           3.22e-21
Time:                        10:18:25   Log-Likelihood:                 2875.1
No. Observations:                2464   AIC:                            -5628.
Df Residuals:                    2403   BIC:                            -5274.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.1646      0.114     -1.440      0.150      -0.389       0.060
sector[T.Consumer Discretionary]     0.0542      0.021      2.581      0.010       0.013       0.095
sector[T.Consumer Staples]           0.0873      0.014      6.280      0.000       0.060       0.115
sector[T.Energy]                     0.1700      0.019      8.806      0.000       0.132       0.208
sector[T.Financials]                 0.1387      0.018      7.508      0.000       0.102       0.175
sector[T.Health Care]                0.0144      0.018      0.817      0.414      -0.020       0.049
sector[T.Industrials]                0.0848      0.016      5.452      0.000       0.054       0.115
sector[T.Information Technology]     0.0665      0.022      3.082      0.002       0.024       0.109
sector[T.Materials]                  0.1076      0.015      6.957      0.000       0.077       0.138
sector[T.Real Estate]                0.1346      0.020      6.575      0.000       0.094       0.175
sector[T.Utilities]                  0.1012      0.016      6.444      0.000       0.070       0.132
ticks[T.AOT TB Equity]              -0.0105      0.015     -0.688      0.491      -0.040       0.019
ticks[T.AP TB Equity]                0.0607      0.012      4.896      0.000       0.036       0.085
ticks[T.BANPU TB Equity]             0.0322      0.012      2.729      0.006       0.009       0.055
ticks[T.BAY TB Equity]               0.0674      0.020      3.332      0.001       0.028       0.107
ticks[T.BBL TB Equity]               0.0728      0.021      3.390      0.001       0.031       0.115
ticks[T.BCH TB Equity]              -0.0053      0.018     -0.302      0.762      -0.040       0.029
ticks[T.BCP TB Equity]               0.0356      0.014      2.621      0.009       0.009       0.062
ticks[T.BDMS TB Equity]              0.0305      0.012      2.456      0.014       0.006       0.055
ticks[T.BEAUTY TB Equity]           -0.1342      0.036     -3.683      0.000      -0.206      -0.063
ticks[T.BEC TB Equity]               0.0124      0.032      0.392      0.695      -0.050       0.074
ticks[T.BEM TB Equity]              -0.0003      0.012     -0.024      0.981      -0.025       0.024
ticks[T.BGRIM TB Equity]            -0.0279      0.015     -1.804      0.071      -0.058       0.002
ticks[T.BH TB Equity]               -0.0108      0.013     -0.825      0.409      -0.036       0.015
ticks[T.BJC TB Equity]               0.0448      0.013      3.489      0.000       0.020       0.070
ticks[T.BTS TB Equity]               0.0533      0.017      3.205      0.001       0.021       0.086
ticks[T.CPALL TB Equity]            -0.0817      0.014     -5.708      0.000      -0.110      -0.054
ticks[T.CPF TB Equity]               0.0997      0.01

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     5.489
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           1.26e-30
Time:                        10:18:50   Log-Likelihood:                 455.25
No. Observations:                 672   AIC:                            -780.5
Df Residuals:                     607   BIC:                            -487.3
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -3.3331      1.188     -2.806      0.005      -5.666      -1.000
sector[T.Consumer Discretionary]     0.5871      0.161      3.641      0.000       0.270       0.904
sector[T.Consumer Staples]           0.5005      0.095      5.289      0.000       0.315       0.686
sector[T.Energy]                     0.9268      0.158      5.870      0.000       0.617       1.237
sector[T.Financials]                 0.4217      0.133      3.169      0.002       0.160       0.683
sector[T.Health Care]                0.4469      0.168      2.663      0.008       0.117       0.776
sector[T.Industrials]                0.5683      0.170      3.337      0.001       0.234       0.903
sector[T.Information Technology]     0.5982      0.149      4.026      0.000       0.306       0.890
sector[T.Materials]                  0.6264      0.106      5.893      0.000       0.418       0.835
sector[T.Real Estate]                0.8251      0.112      7.347      0.000       0.605       1.046
sector[T.Utilities]                  0.4480      0.105      4.265      0.000       0.242       0.654
ticks[T.AOT TB Equity]              -0.1421      0.060     -2.358      0.019      -0.260      -0.024
ticks[T.AP TB Equity]                0.2354      0.043      5.539      0.000       0.152       0.319
ticks[T.AWC TB Equity]              -0.0683      0.136     -0.502      0.616      -0.336       0.199
ticks[T.BANPU TB Equity]             0.2050      0.044      4.708      0.000       0.120       0.291
ticks[T.BAY TB Equity]               0.1495      0.149      1.001      0.317      -0.144       0.443
ticks[T.BBL TB Equity]              -0.0530      0.197     -0.268      0.788      -0.441       0.335
ticks[T.BCH TB Equity]               0.1089      0.088      1.241      0.215      -0.063       0.281
ticks[T.BCP TB Equity]               0.2640      0.077      3.420      0.001       0.112       0.416
ticks[T.BDMS TB Equity]              0.1311      0.042      3.154      0.002       0.049       0.213
ticks[T.BEAUTY TB Equity]            0.2642      0.321      0.824      0.410      -0.365       0.894
ticks[T.BEC TB Equity]               0.9203      0.273      3.367      0.001       0.383       1.457
ticks[T.BEM TB Equity]               0.1118      0.080      1.401      0.162      -0.045       0.269
ticks[T.BGRIM TB Equity]            -0.3463      0.063     -5.463      0.000      -0.471      -0.222
ticks[T.BH TB Equity]                0.2069      0.092      2.242      0.025       0.026       0.388
ticks[T.BJC TB Equity]               0.0096      0.133      0.073      0.942      -0.251       0.271
ticks[T.BTS TB Equity]               0.3995      0.139      2.867      0.004       0.126       0.673
ticks[T.CPALL TB Equity]            -0.3182      0.04

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     2.788
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           4.32e-18
Time:                        10:19:53   Log-Likelihood:                 2080.9
No. Observations:                1682   AIC:                            -3940.
Df Residuals:                    1571   BIC:                            -3337.
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.8183      0.260     -3.145      0.002      -1.329      -0.308
sector[T.Consumer Discretionary]    -0.0693      0.015     -4.703      0.000      -0.098      -0.040
sector[T.Consumer Staples]          -0.0542      0.021     -2.525      0.012      -0.096      -0.012
sector[T.Energy]                     0.0309      0.043      0.715      0.475      -0.054       0.116
sector[T.Financials]                -0.0569      0.070     -0.814      0.415      -0.194       0.080
sector[T.Health Care]               -0.0379      0.013     -2.847      0.004      -0.064      -0.012
sector[T.Industrials]               -0.0645      0.040     -1.628      0.104      -0.142       0.013
sector[T.Information Technology]    -0.0607      0.013     -4.657      0.000      -0.086      -0.035
sector[T.Materials]                  0.0193      0.024      0.800      0.424      -0.028       0.067
sector[T.Real Estate]                0.0861      0.028      3.128      0.002       0.032       0.140
sector[T.Utilities]                 -0.0754      0.036     -2.097      0.036      -0.146      -0.005
ticks[T.ACE TB Equity]               0.0850      0.046      1.866      0.062      -0.004       0.174
ticks[T.ADVANC TB Equity]           -0.2876      0.076     -3.792      0.000      -0.436      -0.139
ticks[T.AMATA TB Equity]            -0.0331      0.022     -1.524      0.128      -0.076       0.009
ticks[T.AOT TB Equity]              -0.2337      0.048     -4.865      0.000      -0.328      -0.139
ticks[T.AP TB Equity]                0.0213      0.017      1.248      0.212      -0.012       0.055
ticks[T.AWC TB Equity]               0.0183      0.040      0.455      0.649      -0.061       0.097
ticks[T.BAFS TB Equity]              0.0447      0.048      0.932      0.351      -0.049       0.139
ticks[T.BAM TB Equity]               0.0081      0.031      0.257      0.797      -0.054       0.070
ticks[T.BANPU TB Equity]            -0.0346      0.031     -1.102      0.271      -0.096       0.027
ticks[T.BAY TB Equity]               0.0338      0.054      0.632      0.527      -0.071       0.139
ticks[T.BBL TB Equity]              -0.0038      0.067     -0.056      0.955      -0.135       0.127
ticks[T.BCH TB Equity]              -0.0156      0.018     -0.852      0.394      -0.052       0.020
ticks[T.BCP TB Equity]               0.0478      0.021      2.288      0.022       0.007       0.089
ticks[T.BDMS TB Equity]             -0.1249      0.048     -2.603      0.009      -0.219      -0.031
ticks[T.BEAUTY TB Equity]            0.0524      0.089      0.589      0.556      -0.122       0.227
ticks[T.BEC TB Equity]              -0.0450      0.025     -1.825      0.068      -0.093       0.003
ticks[T.BEM TB Equity]              -0.0534      0.03

## vol

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_E +  ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     100.7
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:39:55   Log-Likelihood:                 919.06
No. Observations:                2464   AIC:                            -1716.
Df Residuals:                    2403   BIC:                            -1362.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.8379      0.253     15.180      0.000       3.342       4.334
sector[T.Consumer Discretionary]     0.1329      0.046      2.859      0.004       0.042       0.224
sector[T.Consumer Staples]           0.0613      0.031      1.993      0.046       0.001       0.122
sector[T.Energy]                     0.0212      0.043      0.497      0.620      -0.063       0.105
sector[T.Financials]                 0.1644      0.041      4.022      0.000       0.084       0.245
sector[T.Health Care]               -0.0772      0.039     -1.983      0.048      -0.154      -0.001
sector[T.Industrials]               -0.0485      0.034     -1.409      0.159      -0.116       0.019
sector[T.Information Technology]     0.2080      0.048      4.362      0.000       0.115       0.302
sector[T.Materials]                  0.1331      0.034      3.892      0.000       0.066       0.200
sector[T.Real Estate]               -0.1137      0.045     -2.511      0.012      -0.202      -0.025
sector[T.Utilities]                 -0.0389      0.035     -1.119      0.263      -0.107       0.029
ticks[T.AOT TB Equity]               0.0970      0.034      2.876      0.004       0.031       0.163
ticks[T.AP TB Equity]               -0.0200      0.027     -0.728      0.467      -0.074       0.034
ticks[T.BANPU TB Equity]             0.0215      0.026      0.825      0.409      -0.030       0.073
ticks[T.BAY TB Equity]               0.4162      0.045      9.302      0.000       0.328       0.504
ticks[T.BBL TB Equity]              -0.0309      0.047     -0.651      0.515      -0.124       0.062
ticks[T.BCH TB Equity]              -0.1215      0.039     -3.129      0.002      -0.198      -0.045
ticks[T.BCP TB Equity]              -0.3038      0.030    -10.117      0.000      -0.363      -0.245
ticks[T.BDMS TB Equity]             -0.0050      0.027     -0.182      0.856      -0.059       0.049
ticks[T.BEAUTY TB Equity]            0.3103      0.081      3.851      0.000       0.152       0.468
ticks[T.BEC TB Equity]               0.2019      0.070      2.888      0.004       0.065       0.339
ticks[T.BEM TB Equity]              -0.1999      0.028     -7.267      0.000      -0.254      -0.146
ticks[T.BGRIM TB Equity]             0.1420      0.034      4.155      0.000       0.075       0.209
ticks[T.BH TB Equity]                0.0493      0.029      1.708      0.088      -0.007       0.106
ticks[T.BJC TB Equity]               0.1901      0.028      6.696      0.000       0.134       0.246
ticks[T.BTS TB Equity]              -0.4831      0.037    -13.126      0.000      -0.555      -0.411
ticks[T.CPALL TB Equity]            -0.1833      0.032     -5.787      0.000      -0.245      -0.121
ticks[T.CPF TB Equity]               0.0984      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     65.73
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          9.51e-232
Time:                        10:25:21   Log-Likelihood:                 532.02
No. Observations:                 672   AIC:                            -934.0
Df Residuals:                     607   BIC:                            -640.9
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -3.4920      1.060     -3.295      0.001      -5.573      -1.411
sector[T.Consumer Discretionary]     1.2974      0.144      9.019      0.000       1.015       1.580
sector[T.Consumer Staples]           0.4186      0.084      4.959      0.000       0.253       0.584
sector[T.Energy]                     0.7551      0.141      5.362      0.000       0.479       1.032
sector[T.Financials]                -0.6197      0.119     -5.221      0.000      -0.853      -0.387
sector[T.Health Care]                0.9895      0.150      6.611      0.000       0.696       1.283
sector[T.Industrials]                0.9266      0.152      6.099      0.000       0.628       1.225
sector[T.Information Technology]     0.9746      0.133      7.353      0.000       0.714       1.235
sector[T.Materials]                  0.5508      0.095      5.808      0.000       0.365       0.737
sector[T.Real Estate]                0.4964      0.100      4.955      0.000       0.300       0.693
sector[T.Utilities]                  0.7030      0.094      7.502      0.000       0.519       0.887
ticks[T.AOT TB Equity]              -0.0056      0.054     -0.105      0.917      -0.111       0.100
ticks[T.AP TB Equity]                0.1290      0.038      3.403      0.001       0.055       0.203
ticks[T.AWC TB Equity]              -0.6742      0.121     -5.549      0.000      -0.913      -0.436
ticks[T.BANPU TB Equity]             0.1269      0.039      3.267      0.001       0.051       0.203
ticks[T.BAY TB Equity]              -1.1281      0.133     -8.465      0.000      -1.390      -0.866
ticks[T.BBL TB Equity]              -1.3996      0.176     -7.948      0.000      -1.745      -1.054
ticks[T.BCH TB Equity]               0.6743      0.078      8.619      0.000       0.521       0.828
ticks[T.BCP TB Equity]               0.2547      0.069      3.699      0.000       0.119       0.390
ticks[T.BDMS TB Equity]             -0.1905      0.037     -5.138      0.000      -0.263      -0.118
ticks[T.BEAUTY TB Equity]            2.0957      0.286      7.329      0.000       1.534       2.657
ticks[T.BEC TB Equity]               1.8439      0.244      7.561      0.000       1.365       2.323
ticks[T.BEM TB Equity]               0.3572      0.071      5.016      0.000       0.217       0.497
ticks[T.BGRIM TB Equity]             0.0272      0.057      0.482      0.630      -0.084       0.138
ticks[T.BH TB Equity]                0.5057      0.082      6.144      0.000       0.344       0.667
ticks[T.BJC TB Equity]              -0.6556      0.119     -5.529      0.000      -0.888      -0.423
ticks[T.BTS TB Equity]               0.6386      0.124      5.137      0.000       0.394       0.883
ticks[T.CPALL TB Equity]            -0.4210      0.04

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     34.85
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:25:52   Log-Likelihood:                 454.48
No. Observations:                1682   AIC:                            -687.0
Df Residuals:                    1571   BIC:                            -84.49
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.9385      0.684     15.985      0.000       9.596      12.281
sector[T.Consumer Discretionary]     0.5996      0.039     15.470      0.000       0.524       0.676
sector[T.Consumer Staples]           0.6961      0.056     12.338      0.000       0.585       0.807
sector[T.Energy]                     1.7392      0.114     15.304      0.000       1.516       1.962
sector[T.Financials]                 2.2828      0.184     12.429      0.000       1.923       2.643
sector[T.Health Care]                0.1125      0.035      3.208      0.001       0.044       0.181
sector[T.Industrials]                1.3978      0.104     13.418      0.000       1.193       1.602
sector[T.Information Technology]     0.4711      0.034     13.737      0.000       0.404       0.538
sector[T.Materials]                  1.0392      0.063     16.402      0.000       0.915       1.163
sector[T.Real Estate]                0.7879      0.072     10.885      0.000       0.646       0.930
sector[T.Utilities]                  1.2934      0.095     13.670      0.000       1.108       1.479
ticks[T.ACE TB Equity]              -1.4653      0.120    -12.232      0.000      -1.700      -1.230
ticks[T.ADVANC TB Equity]            2.0624      0.199     10.339      0.000       1.671       2.454
ticks[T.AMATA TB Equity]            -0.3086      0.057     -5.408      0.000      -0.421      -0.197
ticks[T.AOT TB Equity]               0.6284      0.126      4.974      0.000       0.381       0.876
ticks[T.AP TB Equity]               -0.1672      0.045     -3.720      0.000      -0.255      -0.079
ticks[T.AWC TB Equity]               1.2319      0.106     11.658      0.000       1.025       1.439
ticks[T.BAFS TB Equity]             -1.1042      0.126     -8.760      0.000      -1.351      -0.857
ticks[T.BAM TB Equity]              -1.1576      0.083    -13.980      0.000      -1.320      -0.995
ticks[T.BANPU TB Equity]             0.9819      0.083     11.876      0.000       0.820       1.144
ticks[T.BAY TB Equity]               2.3677      0.141     16.823      0.000       2.092       2.644
ticks[T.BBL TB Equity]               2.0589      0.176     11.702      0.000       1.714       2.404
ticks[T.BCH TB Equity]              -0.3370      0.048     -6.980      0.000      -0.432      -0.242
ticks[T.BCP TB Equity]               0.5489      0.055      9.986      0.000       0.441       0.657
ticks[T.BDMS TB Equity]              1.3746      0.126     10.892      0.000       1.127       1.622
ticks[T.BEAUTY TB Equity]           -3.1293      0.234    -13.391      0.000      -3.588      -2.671
ticks[T.BEC TB Equity]              -0.5539      0.065     -8.530      0.000      -0.681      -0.427
ticks[T.BEM TB Equity]               0.0209      0.09

## liq

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     85.23
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:36:38   Log-Likelihood:                -1754.0
No. Observations:                2464   AIC:                             3628.
Df Residuals:                    2404   BIC:                             3977.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.9515      0.744      5.314      0.000       2.493       5.410
sector[T.Consumer Discretionary]     1.3003      0.135      9.635      0.000       1.036       1.565
sector[T.Consumer Staples]           0.4770      0.090      5.275      0.000       0.300       0.654
sector[T.Energy]                     0.9600      0.125      7.694      0.000       0.715       1.205
sector[T.Financials]                 0.2820      0.121      2.335      0.020       0.045       0.519
sector[T.Health Care]                0.5623      0.115      4.906      0.000       0.338       0.787
sector[T.Industrials]                0.6598      0.101      6.541      0.000       0.462       0.858
sector[T.Information Technology]     1.0262      0.140      7.354      0.000       0.753       1.300
sector[T.Materials]                  0.3312      0.101      3.282      0.001       0.133       0.529
sector[T.Real Estate]                1.1266      0.132      8.538      0.000       0.868       1.385
sector[T.Utilities]                  0.5640      0.102      5.522      0.000       0.364       0.764
ticks[T.AOT TB Equity]              -0.7905      0.098     -8.028      0.000      -0.984      -0.597
ticks[T.AP TB Equity]                0.4224      0.081      5.237      0.000       0.264       0.581
ticks[T.BANPU TB Equity]             1.1224      0.074     15.235      0.000       0.978       1.267
ticks[T.BAY TB Equity]              -3.8436      0.107    -36.036      0.000      -4.053      -3.634
ticks[T.BBL TB Equity]              -0.3527      0.140     -2.513      0.012      -0.628      -0.078
ticks[T.BCH TB Equity]               1.0307      0.113      9.128      0.000       0.809       1.252
ticks[T.BCP TB Equity]               0.1467      0.089      1.652      0.099      -0.027       0.321
ticks[T.BDMS TB Equity]             -0.4097      0.081     -5.075      0.000      -0.568      -0.251
ticks[T.BEAUTY TB Equity]            1.8273      0.235      7.760      0.000       1.366       2.289
ticks[T.BEC TB Equity]               1.3321      0.205      6.497      0.000       0.930       1.734
ticks[T.BEM TB Equity]               0.2182      0.081      2.684      0.007       0.059       0.378
ticks[T.BGRIM TB Equity]             0.5310      0.101      5.283      0.000       0.334       0.728
ticks[T.BH TB Equity]               -0.0587      0.085     -0.687      0.492      -0.226       0.109
ticks[T.BJC TB Equity]              -0.2719      0.084     -3.243      0.001      -0.436      -0.107
ticks[T.BTS TB Equity]              -0.2201      0.109     -2.023      0.043      -0.433      -0.007
ticks[T.CPALL TB Equity]            -0.5100      0.093     -5.476      0.000      -0.693      -0.327
ticks[T.CPF TB Equity]               0.3314      0.10

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_E + (esg_E*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     31.55
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          1.18e-152
Time:                        10:34:42   Log-Likelihood:                -420.73
No. Observations:                 672   AIC:                             969.5
Df Residuals:                     608   BIC:                             1258.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           15.8161      4.323      3.658      0.000       7.326      24.306
sector[T.Consumer Discretionary]     0.2150      0.593      0.362      0.717      -0.950       1.380
sector[T.Consumer Staples]           0.0757      0.348      0.218      0.828      -0.608       0.759
sector[T.Energy]                     0.3488      0.581      0.601      0.548      -0.792       1.489
sector[T.Financials]                 2.2127      0.481      4.597      0.000       1.267       3.158
sector[T.Health Care]               -1.1160      0.616     -1.813      0.070      -2.325       0.093
sector[T.Industrials]               -0.8237      0.626     -1.316      0.189      -2.053       0.405
sector[T.Information Technology]     0.3976      0.546      0.728      0.467      -0.676       1.471
sector[T.Materials]                 -0.1538      0.391     -0.393      0.694      -0.922       0.614
sector[T.Real Estate]                0.6743      0.412      1.635      0.102      -0.135       1.484
sector[T.Utilities]                  0.0822      0.387      0.213      0.832      -0.677       0.841
ticks[T.AOT TB Equity]              -0.8912      0.219     -4.075      0.000      -1.321      -0.462
ticks[T.AP TB Equity]                0.6005      0.154      3.887      0.000       0.297       0.904
ticks[T.AWC TB Equity]               0.4751      0.501      0.949      0.343      -0.508       1.458
ticks[T.BANPU TB Equity]             1.1763      0.153      7.691      0.000       0.876       1.477
ticks[T.BAY TB Equity]              -2.2896      0.542     -4.226      0.000      -3.354      -1.226
ticks[T.BBL TB Equity]               2.4281      0.720      3.374      0.001       1.015       3.841
ticks[T.BCH TB Equity]              -0.0278      0.323     -0.086      0.931      -0.662       0.606
ticks[T.BCP TB Equity]              -0.8590      0.282     -3.047      0.002      -1.413      -0.305
ticks[T.BDMS TB Equity]             -0.4437      0.152     -2.921      0.004      -0.742      -0.145
ticks[T.BEAUTY TB Equity]           -1.0923      1.179     -0.927      0.354      -3.407       1.222
ticks[T.BEC TB Equity]              -0.1349      1.006     -0.134      0.893      -2.110       1.840
ticks[T.BEM TB Equity]              -0.6274      0.293     -2.144      0.032      -1.202      -0.053
ticks[T.BGRIM TB Equity]             0.6531      0.232      2.819      0.005       0.198       1.108
ticks[T.BH TB Equity]               -0.6445      0.339     -1.904      0.057      -1.309       0.020
ticks[T.BJC TB Equity]               1.5507      0.485      3.198      0.001       0.598       2.503
ticks[T.BTS TB Equity]              -1.2592      0.510     -2.468      0.014      -2.261      -0.257
ticks[T.CPALL TB Equity]            -0.2474      0.18

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_E + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     62.09
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:34:15   Log-Likelihood:                -1097.8
No. Observations:                1682   AIC:                             2416.
Df Residuals:                    1572   BIC:                             3013.
Df Model:                         109                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.4088      1.701      6.118      0.000       7.072      13.746
sector[T.Consumer Discretionary]     0.2496      0.097      2.565      0.010       0.059       0.440
sector[T.Consumer Staples]           0.3497      0.142      2.469      0.014       0.072       0.628
sector[T.Energy]                     2.3432      0.280      8.377      0.000       1.795       2.892
sector[T.Financials]                 1.8268      0.460      3.974      0.000       0.925       2.729
sector[T.Health Care]                0.0724      0.088      0.821      0.412      -0.101       0.245
sector[T.Industrials]                0.6039      0.262      2.308      0.021       0.091       1.117
sector[T.Information Technology]     0.6796      0.085      8.038      0.000       0.514       0.845
sector[T.Materials]                  0.3026      0.159      1.901      0.057      -0.010       0.615
sector[T.Real Estate]                1.0382      0.180      5.761      0.000       0.685       1.392
sector[T.Utilities]                  0.8976      0.237      3.788      0.000       0.433       1.362
ticks[T.ACE TB Equity]              -0.8327      0.301     -2.770      0.006      -1.422      -0.243
ticks[T.ADVANC TB Equity]           -0.1529      0.502     -0.305      0.761      -1.137       0.831
ticks[T.AMATA TB Equity]             1.0174      0.141      7.204      0.000       0.740       1.294
ticks[T.AOT TB Equity]              -1.2596      0.316     -3.983      0.000      -1.880      -0.639
ticks[T.AP TB Equity]                1.2029      0.109     11.042      0.000       0.989       1.417
ticks[T.AWC TB Equity]               0.3563      0.266      1.341      0.180      -0.165       0.877
ticks[T.BAFS TB Equity]             -1.4367      0.315     -4.561      0.000      -2.055      -0.819
ticks[T.BAM TB Equity]               0.3590      0.208      1.725      0.085      -0.049       0.767
ticks[T.BANPU TB Equity]             1.9718      0.202      9.763      0.000       1.576       2.368
ticks[T.BAY TB Equity]              -1.6581      0.352     -4.716      0.000      -2.348      -0.969
ticks[T.BBL TB Equity]               2.6235      0.438      5.994      0.000       1.765       3.482
ticks[T.BCH TB Equity]               0.5208      0.121      4.313      0.000       0.284       0.758
ticks[T.BCP TB Equity]               0.4071      0.138      2.952      0.003       0.137       0.678
ticks[T.BDMS TB Equity]              0.0641      0.317      0.202      0.840      -0.559       0.687
ticks[T.BEAUTY TB Equity]           -1.2502      0.587     -2.130      0.033      -2.402      -0.099
ticks[T.BEC TB Equity]               0.1944      0.163      1.190      0.234      -0.126       0.515
ticks[T.BEM TB Equity]               0.0613      0.25

# social

## return

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     3.340
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           3.34e-28
Time:                        11:00:40   Log-Likelihood:                 4808.0
No. Observations:                4818   AIC:                            -9392.
Df Residuals:                    4706   BIC:                            -8666.
Df Model:                         111                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.2569      0.055     -4.706      0.000      -0.364      -0.150
sector[T.Consumer Discretionary]    -0.0115      0.007     -1.724      0.085      -0.025       0.002
sector[T.Consumer Staples]          -0.0108      0.008     -1.293      0.196      -0.027       0.006
sector[T.Energy]                     0.0464      0.012      3.870      0.000       0.023       0.070
sector[T.Financials]                 0.0277      0.015      1.837      0.066      -0.002       0.057
sector[T.Health Care]               -0.0131      0.009     -1.521      0.128      -0.030       0.004
sector[T.Industrials]               -0.0170      0.031     -0.551      0.581      -0.077       0.043
sector[T.Information Technology]    -0.0180      0.009     -1.931      0.054      -0.036       0.000
sector[T.Materials]                  0.0355      0.009      4.092      0.000       0.019       0.053
sector[T.Real Estate]                0.0668      0.008      7.885      0.000       0.050       0.083
sector[T.Utilities]                 -0.0023      0.009     -0.257      0.797      -0.020       0.015
ticks[T.ACE TB Equity]              -0.0034      0.028     -0.118      0.906      -0.059       0.052
ticks[T.ADVANC TB Equity]           -0.0748      0.019     -3.903      0.000      -0.112      -0.037
ticks[T.AMATA TB Equity]            -0.0253      0.025     -0.996      0.319      -0.075       0.024
ticks[T.AOT TB Equity]              -0.0334      0.035     -0.965      0.335      -0.101       0.034
ticks[T.AP TB Equity]                0.0087      0.011      0.814      0.416      -0.012       0.030
ticks[T.AWC TB Equity]               0.0511      0.017      2.955      0.003       0.017       0.085
ticks[T.BAFS TB Equity]              0.0231      0.041      0.569      0.570      -0.057       0.103
ticks[T.BAM TB Equity]              -0.0332      0.025     -1.304      0.192      -0.083       0.017
ticks[T.BANPU TB Equity]            -0.0149      0.013     -1.116      0.264      -0.041       0.011
ticks[T.BAY TB Equity]               0.1037      0.017      5.980      0.000       0.070       0.138
ticks[T.BBL TB Equity]               0.0361      0.017      2.127      0.033       0.003       0.069
ticks[T.BCH TB Equity]              -0.0365      0.012     -2.956      0.003      -0.061      -0.012
ticks[T.BCP TB Equity]               0.0177      0.012      1.478      0.140      -0.006       0.041
ticks[T.BDMS TB Equity]             -0.0254      0.015     -1.703      0.089      -0.055       0.004
ticks[T.BEAUTY TB Equity]           -0.1073      0.021     -5.196      0.000      -0.148      -0.067
ticks[T.BEC TB Equity]              -0.0498      0.011     -4.497      0.000      -0.071      -0.028
ticks[T.BEM TB Equity]              -0.0118      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     4.037
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           2.62e-22
Time:                        10:38:51   Log-Likelihood:                 2878.5
No. Observations:                2464   AIC:                            -5635.
Df Residuals:                    2403   BIC:                            -5281.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.1963      0.114     -1.716      0.086      -0.421       0.028
sector[T.Consumer Discretionary]     0.0652      0.021      3.043      0.002       0.023       0.107
sector[T.Consumer Staples]           0.0793      0.014      5.589      0.000       0.051       0.107
sector[T.Energy]                     0.1729      0.018      9.378      0.000       0.137       0.209
sector[T.Financials]                 0.1381      0.018      7.789      0.000       0.103       0.173
sector[T.Health Care]                0.0104      0.018      0.588      0.557      -0.024       0.045
sector[T.Industrials]                0.0806      0.015      5.332      0.000       0.051       0.110
sector[T.Information Technology]     0.0674      0.021      3.149      0.002       0.025       0.109
sector[T.Materials]                  0.1043      0.015      6.892      0.000       0.075       0.134
sector[T.Real Estate]                0.1509      0.021      7.195      0.000       0.110       0.192
sector[T.Utilities]                  0.1074      0.016      6.768      0.000       0.076       0.139
ticks[T.AOT TB Equity]              -0.0090      0.015     -0.593      0.553      -0.039       0.021
ticks[T.AP TB Equity]                0.0659      0.012      5.296      0.000       0.042       0.090
ticks[T.BANPU TB Equity]             0.0183      0.013      1.427      0.154      -0.007       0.043
ticks[T.BAY TB Equity]               0.0600      0.020      2.942      0.003       0.020       0.100
ticks[T.BBL TB Equity]               0.0698      0.021      3.316      0.001       0.029       0.111
ticks[T.BCH TB Equity]               0.0009      0.018      0.051      0.959      -0.034       0.036
ticks[T.BCP TB Equity]               0.0437      0.014      3.149      0.002       0.016       0.071
ticks[T.BDMS TB Equity]              0.0229      0.013      1.806      0.071      -0.002       0.048
ticks[T.BEAUTY TB Equity]           -0.1201      0.036     -3.329      0.001      -0.191      -0.049
ticks[T.BEC TB Equity]               0.0148      0.032      0.469      0.639      -0.047       0.077
ticks[T.BEM TB Equity]               0.0048      0.013      0.382      0.703      -0.020       0.029
ticks[T.BGRIM TB Equity]            -0.0158      0.015     -1.045      0.296      -0.045       0.014
ticks[T.BH TB Equity]               -0.0135      0.013     -1.035      0.301      -0.039       0.012
ticks[T.BJC TB Equity]               0.0389      0.013      2.992      0.003       0.013       0.064
ticks[T.BTS TB Equity]               0.0356      0.011      3.136      0.002       0.013       0.058
ticks[T.CPALL TB Equity]            -0.0793      0.014     -5.557      0.000      -0.107      -0.051
ticks[T.CPF TB Equity]               0.0945      0.01

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     5.540
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           5.45e-31
Time:                        10:39:01   Log-Likelihood:                 456.38
No. Observations:                 672   AIC:                            -782.8
Df Residuals:                     607   BIC:                            -489.6
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -3.2125      1.112     -2.889      0.004      -5.397      -1.028
sector[T.Consumer Discretionary]     0.6643      0.200      3.316      0.001       0.271       1.058
sector[T.Consumer Staples]           0.3456      0.052      6.687      0.000       0.244       0.447
sector[T.Energy]                     0.8976      0.163      5.504      0.000       0.577       1.218
sector[T.Financials]                 0.5577      0.086      6.519      0.000       0.390       0.726
sector[T.Health Care]                0.3521      0.115      3.073      0.002       0.127       0.577
sector[T.Industrials]                0.4502      0.121      3.733      0.000       0.213       0.687
sector[T.Information Technology]     0.5920      0.137      4.324      0.000       0.323       0.861
sector[T.Materials]                  0.5911      0.097      6.064      0.000       0.400       0.783
sector[T.Real Estate]                0.9996      0.193      5.188      0.000       0.621       1.378
sector[T.Utilities]                  0.4893      0.131      3.742      0.000       0.232       0.746
ticks[T.AOT TB Equity]              -0.1101      0.064     -1.717      0.086      -0.236       0.016
ticks[T.AP TB Equity]                0.2296      0.041      5.640      0.000       0.150       0.310
ticks[T.AWC TB Equity]               0.3984      0.142      2.807      0.005       0.120       0.677
ticks[T.BANPU TB Equity]            -0.0409      0.141     -0.291      0.771      -0.318       0.236
ticks[T.BAY TB Equity]               0.1170      0.166      0.703      0.482      -0.210       0.444
ticks[T.BBL TB Equity]               0.0063      0.166      0.038      0.970      -0.320       0.332
ticks[T.BCH TB Equity]               0.0991      0.069      1.444      0.149      -0.036       0.234
ticks[T.BCP TB Equity]               0.4065      0.154      2.643      0.008       0.104       0.709
ticks[T.BDMS TB Equity]              0.1069      0.039      2.769      0.006       0.031       0.183
ticks[T.BEAUTY TB Equity]            0.2904      0.319      0.910      0.363      -0.336       0.917
ticks[T.BEC TB Equity]               0.8700      0.229      3.796      0.000       0.420       1.320
ticks[T.BEM TB Equity]              -0.0155      0.043     -0.363      0.717      -0.100       0.069
ticks[T.BGRIM TB Equity]            -0.2060      0.044     -4.732      0.000      -0.291      -0.120
ticks[T.BH TB Equity]                0.1461      0.061      2.376      0.018       0.025       0.267
ticks[T.BJC TB Equity]               0.0648      0.121      0.536      0.592      -0.173       0.303
ticks[T.BTS TB Equity]               0.1683      0.053      3.150      0.002       0.063       0.273
ticks[T.CPALL TB Equity]            -0.3047      0.04

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     2.788
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           4.38e-18
Time:                        10:39:14   Log-Likelihood:                 2080.9
No. Observations:                1682   AIC:                            -3940.
Df Residuals:                    1571   BIC:                            -3337.
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.8188      0.261     -3.143      0.002      -1.330      -0.308
sector[T.Consumer Discretionary]    -0.0690      0.015     -4.695      0.000      -0.098      -0.040
sector[T.Consumer Staples]          -0.0618      0.024     -2.576      0.010      -0.109      -0.015
sector[T.Energy]                     0.0293      0.043      0.678      0.498      -0.056       0.114
sector[T.Financials]                -0.0513      0.068     -0.754      0.451      -0.185       0.082
sector[T.Health Care]               -0.0433      0.013     -3.232      0.001      -0.070      -0.017
sector[T.Industrials]               -0.0615      0.040     -1.543      0.123      -0.140       0.017
sector[T.Information Technology]    -0.0674      0.016     -4.094      0.000      -0.100      -0.035
sector[T.Materials]                  0.0215      0.024      0.890      0.374      -0.026       0.069
sector[T.Real Estate]                0.0960      0.025      3.818      0.000       0.047       0.145
sector[T.Utilities]                 -0.0748      0.036     -2.083      0.037      -0.145      -0.004
ticks[T.ACE TB Equity]               0.0868      0.046      1.893      0.059      -0.003       0.177
ticks[T.ADVANC TB Equity]           -0.2906      0.076     -3.802      0.000      -0.441      -0.141
ticks[T.AMATA TB Equity]            -0.0182      0.030     -0.603      0.546      -0.077       0.041
ticks[T.AOT TB Equity]              -0.2421      0.050     -4.821      0.000      -0.341      -0.144
ticks[T.AP TB Equity]                0.0262      0.016      1.655      0.098      -0.005       0.057
ticks[T.AWC TB Equity]               0.0212      0.039      0.543      0.588      -0.055       0.098
ticks[T.BAFS TB Equity]              0.0372      0.045      0.834      0.405      -0.050       0.125
ticks[T.BAM TB Equity]               0.0061      0.031      0.195      0.845      -0.055       0.068
ticks[T.BANPU TB Equity]            -0.0428      0.034     -1.245      0.213      -0.110       0.025
ticks[T.BAY TB Equity]               0.0304      0.054      0.558      0.577      -0.076       0.137
ticks[T.BBL TB Equity]              -0.0075      0.068     -0.109      0.913      -0.141       0.127
ticks[T.BCH TB Equity]              -0.0180      0.019     -0.943      0.346      -0.056       0.019
ticks[T.BCP TB Equity]               0.0441      0.019      2.282      0.023       0.006       0.082
ticks[T.BDMS TB Equity]             -0.1236      0.048     -2.584      0.010      -0.217      -0.030
ticks[T.BEAUTY TB Equity]            0.0596      0.091      0.659      0.510      -0.118       0.237
ticks[T.BEC TB Equity]              -0.0467      0.025     -1.860      0.063      -0.096       0.003
ticks[T.BEM TB Equity]              -0.0723      0.04

## vol

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     103.9
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:40:29   Log-Likelihood:                 946.69
No. Observations:                2464   AIC:                            -1771.
Df Residuals:                    2403   BIC:                            -1417.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.3267      0.251     13.272      0.000       2.835       3.818
sector[T.Consumer Discretionary]     0.2525      0.047      5.384      0.000       0.161       0.344
sector[T.Consumer Staples]          -0.0015      0.031     -0.048      0.962      -0.062       0.059
sector[T.Energy]                     0.1686      0.040      4.176      0.000       0.089       0.248
sector[T.Financials]                 0.0534      0.039      1.374      0.169      -0.023       0.130
sector[T.Health Care]               -0.0985      0.039     -2.551      0.011      -0.174      -0.023
sector[T.Industrials]               -0.0107      0.033     -0.324      0.746      -0.076       0.054
sector[T.Information Technology]     0.1688      0.047      3.600      0.000       0.077       0.261
sector[T.Materials]                  0.1680      0.033      5.071      0.000       0.103       0.233
sector[T.Real Estate]               -0.0351      0.046     -0.763      0.445      -0.125       0.055
sector[T.Utilities]                  0.0200      0.035      0.576      0.565      -0.048       0.088
ticks[T.AOT TB Equity]               0.1553      0.033      4.696      0.000       0.090       0.220
ticks[T.AP TB Equity]                0.0003      0.027      0.011      0.991      -0.053       0.054
ticks[T.BANPU TB Equity]            -0.0825      0.028     -2.936      0.003      -0.138      -0.027
ticks[T.BAY TB Equity]               0.3347      0.045      7.490      0.000       0.247       0.422
ticks[T.BBL TB Equity]              -0.1508      0.046     -3.274      0.001      -0.241      -0.060
ticks[T.BCH TB Equity]              -0.0818      0.039     -2.116      0.034      -0.158      -0.006
ticks[T.BCP TB Equity]              -0.1972      0.030     -6.494      0.000      -0.257      -0.138
ticks[T.BDMS TB Equity]             -0.0630      0.028     -2.266      0.024      -0.118      -0.008
ticks[T.BEAUTY TB Equity]            0.3044      0.079      3.853      0.000       0.149       0.459
ticks[T.BEC TB Equity]               0.2123      0.069      3.071      0.002       0.077       0.348
ticks[T.BEM TB Equity]              -0.1683      0.027     -6.126      0.000      -0.222      -0.114
ticks[T.BGRIM TB Equity]             0.1483      0.033      4.486      0.000       0.083       0.213
ticks[T.BH TB Equity]                0.0464      0.029      1.627      0.104      -0.010       0.102
ticks[T.BJC TB Equity]               0.1107      0.028      3.892      0.000       0.055       0.167
ticks[T.BTS TB Equity]              -0.3767      0.025    -15.166      0.000      -0.425      -0.328
ticks[T.CPALL TB Equity]            -0.1859      0.031     -5.949      0.000      -0.247      -0.125
ticks[T.CPF TB Equity]               0.0601      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     69.41
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          5.83e-238
Time:                        10:40:52   Log-Likelihood:                 548.09
No. Observations:                 672   AIC:                            -966.2
Df Residuals:                     607   BIC:                            -673.0
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -3.1439      0.970     -3.240      0.001      -5.049      -1.238
sector[T.Consumer Discretionary]     1.5477      0.175      8.856      0.000       1.204       1.891
sector[T.Consumer Staples]          -0.0072      0.045     -0.159      0.874      -0.096       0.081
sector[T.Energy]                     0.9001      0.142      6.326      0.000       0.621       1.180
sector[T.Financials]                -0.2793      0.075     -3.742      0.000      -0.426      -0.133
sector[T.Health Care]                0.6958      0.100      6.960      0.000       0.499       0.892
sector[T.Industrials]                0.6993      0.105      6.647      0.000       0.493       0.906
sector[T.Information Technology]     0.8812      0.119      7.379      0.000       0.647       1.116
sector[T.Materials]                  0.5483      0.085      6.447      0.000       0.381       0.715
sector[T.Real Estate]                1.0175      0.168      6.053      0.000       0.687       1.348
sector[T.Utilities]                  0.8967      0.114      7.860      0.000       0.673       1.121
ticks[T.AOT TB Equity]               0.1425      0.056      2.548      0.011       0.033       0.252
ticks[T.AP TB Equity]                0.0650      0.036      1.830      0.068      -0.005       0.135
ticks[T.AWC TB Equity]               0.7345      0.124      5.932      0.000       0.491       0.978
ticks[T.BANPU TB Equity]            -0.6732      0.123     -5.478      0.000      -0.915      -0.432
ticks[T.BAY TB Equity]              -1.2594      0.145     -8.671      0.000      -1.545      -0.974
ticks[T.BBL TB Equity]              -1.2478      0.145     -8.620      0.000      -1.532      -0.964
ticks[T.BCH TB Equity]               0.5362      0.060      8.952      0.000       0.419       0.654
ticks[T.BCP TB Equity]               0.7363      0.134      5.487      0.000       0.473       1.000
ticks[T.BDMS TB Equity]             -0.1311      0.034     -3.894      0.000      -0.197      -0.065
ticks[T.BEAUTY TB Equity]            2.0997      0.278      7.543      0.000       1.553       2.646
ticks[T.BEC TB Equity]               1.5691      0.200      7.847      0.000       1.176       1.962
ticks[T.BEM TB Equity]               0.0282      0.037      0.754      0.451      -0.045       0.101
ticks[T.BGRIM TB Equity]             0.3468      0.038      9.132      0.000       0.272       0.421
ticks[T.BH TB Equity]                0.2907      0.054      5.420      0.000       0.185       0.396
ticks[T.BJC TB Equity]              -0.6519      0.106     -6.173      0.000      -0.859      -0.444
ticks[T.BTS TB Equity]               0.1857      0.047      3.984      0.000       0.094       0.277
ticks[T.CPALL TB Equity]            -0.3120      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     34.85
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:54:28   Log-Likelihood:                 454.51
No. Observations:                1682   AIC:                            -687.0
Df Residuals:                    1571   BIC:                            -84.54
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.9198      0.685     15.937      0.000       9.576      12.264
sector[T.Consumer Discretionary]     0.6028      0.039     15.600      0.000       0.527       0.679
sector[T.Consumer Staples]           0.6076      0.063      9.638      0.000       0.484       0.731
sector[T.Energy]                     1.7210      0.114     15.112      0.000       1.498       1.944
sector[T.Financials]                 2.3413      0.179     13.083      0.000       1.990       2.692
sector[T.Health Care]                0.0509      0.035      1.442      0.149      -0.018       0.120
sector[T.Industrials]                1.4327      0.105     13.676      0.000       1.227       1.638
sector[T.Information Technology]     0.3933      0.043      9.089      0.000       0.308       0.478
sector[T.Materials]                  1.0643      0.063     16.775      0.000       0.940       1.189
sector[T.Real Estate]                0.9003      0.066     13.610      0.000       0.771       1.030
sector[T.Utilities]                  1.3000      0.094     13.769      0.000       1.115       1.485
ticks[T.ACE TB Equity]              -1.4428      0.121    -11.957      0.000      -1.679      -1.206
ticks[T.ADVANC TB Equity]            2.0251      0.201     10.073      0.000       1.631       2.419
ticks[T.AMATA TB Equity]            -0.1348      0.079     -1.702      0.089      -0.290       0.021
ticks[T.AOT TB Equity]               0.5293      0.132      4.008      0.000       0.270       0.788
ticks[T.AP TB Equity]               -0.1122      0.042     -2.694      0.007      -0.194      -0.031
ticks[T.AWC TB Equity]               1.2610      0.103     12.279      0.000       1.060       1.462
ticks[T.BAFS TB Equity]             -1.1848      0.117    -10.089      0.000      -1.415      -0.954
ticks[T.BAM TB Equity]              -1.1798      0.083    -14.285      0.000      -1.342      -1.018
ticks[T.BANPU TB Equity]             0.8856      0.090      9.786      0.000       0.708       1.063
ticks[T.BAY TB Equity]               2.3253      0.143     16.234      0.000       2.044       2.606
ticks[T.BBL TB Equity]               2.0116      0.180     11.200      0.000       1.659       2.364
ticks[T.BCH TB Equity]              -0.3656      0.050     -7.276      0.000      -0.464      -0.267
ticks[T.BCP TB Equity]               0.5086      0.051      9.999      0.000       0.409       0.608
ticks[T.BDMS TB Equity]              1.3882      0.126     11.035      0.000       1.141       1.635
ticks[T.BEAUTY TB Equity]           -3.0418      0.238    -12.780      0.000      -3.509      -2.575
ticks[T.BEC TB Equity]              -0.5745      0.066     -8.695      0.000      -0.704      -0.445
ticks[T.BEM TB Equity]              -0.1967      0.10

## liq

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     85.10
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:54:43   Log-Likelihood:                -1755.2
No. Observations:                2464   AIC:                             3630.
Df Residuals:                    2404   BIC:                             3979.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.8065      0.746      5.101      0.000       2.343       5.270
sector[T.Consumer Discretionary]     1.3232      0.138      9.607      0.000       1.053       1.593
sector[T.Consumer Staples]           0.4745      0.093      5.128      0.000       0.293       0.656
sector[T.Energy]                     1.0275      0.119      8.635      0.000       0.794       1.261
sector[T.Financials]                 0.2262      0.116      1.948      0.052      -0.002       0.454
sector[T.Health Care]                0.5669      0.115      4.931      0.000       0.341       0.792
sector[T.Industrials]                0.6961      0.098      7.096      0.000       0.504       0.888
sector[T.Information Technology]     1.0030      0.139      7.225      0.000       0.731       1.275
sector[T.Materials]                  0.3623      0.099      3.663      0.000       0.168       0.556
sector[T.Real Estate]                1.1073      0.136      8.163      0.000       0.841       1.373
sector[T.Utilities]                  0.5722      0.103      5.536      0.000       0.370       0.775
ticks[T.AOT TB Equity]              -0.7665      0.098     -7.843      0.000      -0.958      -0.575
ticks[T.AP TB Equity]                0.4139      0.081      5.100      0.000       0.255       0.573
ticks[T.BANPU TB Equity]             1.1213      0.081     13.858      0.000       0.963       1.280
ticks[T.BAY TB Equity]              -3.8623      0.108    -35.736      0.000      -4.074      -3.650
ticks[T.BBL TB Equity]              -0.4045      0.138     -2.939      0.003      -0.674      -0.135
ticks[T.BCH TB Equity]               1.0291      0.114      9.040      0.000       0.806       1.252
ticks[T.BCP TB Equity]               0.1724      0.091      1.898      0.058      -0.006       0.350
ticks[T.BDMS TB Equity]             -0.4122      0.083     -4.978      0.000      -0.575      -0.250
ticks[T.BEAUTY TB Equity]            1.7726      0.234      7.586      0.000       1.314       2.231
ticks[T.BEC TB Equity]               1.3297      0.205      6.480      0.000       0.927       1.732
ticks[T.BEM TB Equity]               0.2158      0.082      2.629      0.009       0.055       0.377
ticks[T.BGRIM TB Equity]             0.4892      0.098      4.968      0.000       0.296       0.682
ticks[T.BH TB Equity]               -0.0501      0.085     -0.587      0.557      -0.217       0.117
ticks[T.BJC TB Equity]              -0.2915      0.085     -3.431      0.001      -0.458      -0.125
ticks[T.BTS TB Equity]              -0.0974      0.074     -1.311      0.190      -0.243       0.048
ticks[T.CPALL TB Equity]            -0.5211      0.093     -5.607      0.000      -0.703      -0.339
ticks[T.CPF TB Equity]               0.3310      0.10

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_S + (esg_S*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     31.53
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          1.35e-152
Time:                        10:54:55   Log-Likelihood:                -420.89
No. Observations:                 672   AIC:                             969.8
Df Residuals:                     608   BIC:                             1258.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           15.2896      4.053      3.773      0.000       7.331      23.248
sector[T.Consumer Discretionary]    -0.2203      0.738     -0.298      0.766      -1.670       1.230
sector[T.Consumer Staples]           0.8890      0.187      4.752      0.000       0.522       1.256
sector[T.Energy]                     0.3164      0.601      0.526      0.599      -0.864       1.497
sector[T.Financials]                 1.5115      0.309      4.886      0.000       0.904       2.119
sector[T.Health Care]               -0.5846      0.422     -1.386      0.166      -1.413       0.244
sector[T.Industrials]               -0.2776      0.444     -0.625      0.532      -1.150       0.595
sector[T.Information Technology]     0.4954      0.504      0.983      0.326      -0.495       1.486
sector[T.Materials]                 -0.0469      0.359     -0.130      0.896      -0.753       0.659
sector[T.Real Estate]               -0.2814      0.710     -0.396      0.692      -1.676       1.113
sector[T.Utilities]                 -0.1993      0.482     -0.413      0.679      -1.146       0.747
ticks[T.AOT TB Equity]              -1.1040      0.232     -4.757      0.000      -1.560      -0.648
ticks[T.AP TB Equity]                0.6671      0.148      4.519      0.000       0.377       0.957
ticks[T.AWC TB Equity]              -2.1291      0.516     -4.126      0.000      -3.143      -1.116
ticks[T.BANPU TB Equity]             2.5680      0.509      5.048      0.000       1.569       3.567
ticks[T.BAY TB Equity]              -2.0966      0.608     -3.450      0.001      -3.290      -0.903
ticks[T.BBL TB Equity]               2.1134      0.606      3.490      0.001       0.924       3.303
ticks[T.BCH TB Equity]               0.1177      0.253      0.465      0.642      -0.379       0.615
ticks[T.BCP TB Equity]              -1.6801      0.563     -2.985      0.003      -2.785      -0.575
ticks[T.BDMS TB Equity]             -0.4204      0.141     -2.976      0.003      -0.698      -0.143
ticks[T.BEAUTY TB Equity]           -1.1502      1.175     -0.979      0.328      -3.458       1.158
ticks[T.BEC TB Equity]               0.2496      0.845      0.295      0.768      -1.410       1.909
ticks[T.BEM TB Equity]               0.0279      0.158      0.177      0.859      -0.282       0.338
ticks[T.BGRIM TB Equity]            -0.0289      0.160     -0.180      0.857      -0.344       0.286
ticks[T.BH TB Equity]               -0.2819      0.226     -1.245      0.213      -0.726       0.163
ticks[T.BJC TB Equity]               1.3726      0.443      3.100      0.002       0.503       2.242
ticks[T.BTS TB Equity]              -0.1900      0.197     -0.965      0.335      -0.577       0.197
ticks[T.CPALL TB Equity]            -0.3749      0.16

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_S + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     62.07
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:55:20   Log-Likelihood:                -1098.0
No. Observations:                1682   AIC:                             2416.
Df Residuals:                    1572   BIC:                             3013.
Df Model:                         109                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           10.4774      1.704      6.150      0.000       7.136      13.819
sector[T.Consumer Discretionary]     0.2541      0.097      2.619      0.009       0.064       0.444
sector[T.Consumer Staples]           0.3194      0.158      2.016      0.044       0.009       0.630
sector[T.Energy]                     2.3388      0.280      8.341      0.000       1.789       2.889
sector[T.Financials]                 1.8848      0.448      4.210      0.000       1.007       2.763
sector[T.Health Care]                0.0448      0.089      0.505      0.614      -0.129       0.219
sector[T.Industrials]                0.6160      0.263      2.341      0.019       0.100       1.132
sector[T.Information Technology]     0.6542      0.108      6.079      0.000       0.443       0.865
sector[T.Materials]                  0.3136      0.159      1.967      0.049       0.001       0.626
sector[T.Real Estate]                1.0975      0.164      6.687      0.000       0.776       1.419
sector[T.Utilities]                  0.9071      0.236      3.836      0.000       0.443       1.371
ticks[T.ACE TB Equity]              -0.8326      0.303     -2.749      0.006      -1.427      -0.239
ticks[T.ADVANC TB Equity]           -0.1515      0.506     -0.300      0.765      -1.144       0.841
ticks[T.AMATA TB Equity]             1.0775      0.197      5.459      0.000       0.690       1.465
ticks[T.AOT TB Equity]              -1.2885      0.331     -3.896      0.000      -1.937      -0.640
ticks[T.AP TB Equity]                1.2335      0.100     12.322      0.000       1.037       1.430
ticks[T.AWC TB Equity]               0.3922      0.258      1.519      0.129      -0.114       0.899
ticks[T.BAFS TB Equity]             -1.5014      0.293     -5.124      0.000      -2.076      -0.927
ticks[T.BAM TB Equity]               0.3473      0.208      1.673      0.095      -0.060       0.755
ticks[T.BANPU TB Equity]             1.9438      0.222      8.742      0.000       1.508       2.380
ticks[T.BAY TB Equity]              -1.6596      0.358     -4.636      0.000      -2.362      -0.957
ticks[T.BBL TB Equity]               2.6325      0.447      5.890      0.000       1.756       3.509
ticks[T.BCH TB Equity]               0.5153      0.126      4.098      0.000       0.269       0.762
ticks[T.BCP TB Equity]               0.3775      0.128      2.957      0.003       0.127       0.628
ticks[T.BDMS TB Equity]              0.0783      0.317      0.247      0.805      -0.543       0.699
ticks[T.BEAUTY TB Equity]           -1.2362      0.598     -2.067      0.039      -2.409      -0.063
ticks[T.BEC TB Equity]               0.1929      0.166      1.161      0.246      -0.133       0.519
ticks[T.BEM TB Equity]              -0.0297      0.26

# governance

## return

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     3.325
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           5.71e-28
Time:                        11:03:36   Log-Likelihood:                 4807.2
No. Observations:                4818   AIC:                            -9390.
Df Residuals:                    4706   BIC:                            -8665.
Df Model:                         111                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.2424      0.056     -4.293      0.000      -0.353      -0.132
sector[T.Consumer Discretionary]    -0.0115      0.007     -1.681      0.093      -0.025       0.002
sector[T.Consumer Staples]          -0.0074      0.008     -0.941      0.347      -0.023       0.008
sector[T.Energy]                     0.0467      0.013      3.725      0.000       0.022       0.071
sector[T.Financials]                 0.0326      0.015      2.215      0.027       0.004       0.061
sector[T.Health Care]               -0.0105      0.008     -1.258      0.209      -0.027       0.006
sector[T.Industrials]               -0.0237      0.031     -0.769      0.442      -0.084       0.037
sector[T.Information Technology]    -0.0144      0.009     -1.619      0.106      -0.032       0.003
sector[T.Materials]                  0.0355      0.009      4.080      0.000       0.018       0.053
sector[T.Real Estate]                0.0656      0.008      7.792      0.000       0.049       0.082
sector[T.Utilities]                 -0.0007      0.010     -0.072      0.943      -0.021       0.019
ticks[T.ACE TB Equity]              -0.0032      0.029     -0.113      0.910      -0.059       0.053
ticks[T.ADVANC TB Equity]           -0.0725      0.020     -3.719      0.000      -0.111      -0.034
ticks[T.AMATA TB Equity]            -0.0336      0.025     -1.335      0.182      -0.083       0.016
ticks[T.AOT TB Equity]              -0.0237      0.036     -0.654      0.513      -0.095       0.047
ticks[T.AP TB Equity]                0.0089      0.011      0.821      0.412      -0.012       0.030
ticks[T.AWC TB Equity]               0.0503      0.018      2.801      0.005       0.015       0.086
ticks[T.BAFS TB Equity]              0.0220      0.041      0.537      0.591      -0.058       0.102
ticks[T.BAM TB Equity]              -0.0338      0.025     -1.327      0.185      -0.084       0.016
ticks[T.BANPU TB Equity]            -0.0109      0.013     -0.837      0.403      -0.036       0.015
ticks[T.BAY TB Equity]               0.1064      0.017      6.184      0.000       0.073       0.140
ticks[T.BBL TB Equity]               0.0387      0.017      2.218      0.027       0.004       0.073
ticks[T.BCH TB Equity]              -0.0393      0.014     -2.794      0.005      -0.067      -0.012
ticks[T.BCP TB Equity]               0.0144      0.012      1.233      0.218      -0.009       0.037
ticks[T.BDMS TB Equity]             -0.0236      0.015     -1.584      0.113      -0.053       0.006
ticks[T.BEAUTY TB Equity]           -0.1069      0.022     -4.960      0.000      -0.149      -0.065
ticks[T.BEC TB Equity]              -0.0460      0.011     -4.233      0.000      -0.067      -0.025
ticks[T.BEM TB Equity]              -0.0056      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     3.876
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           7.27e-21
Time:                        11:03:50   Log-Likelihood:                 2874.0
No. Observations:                2464   AIC:                            -5626.
Df Residuals:                    2403   BIC:                            -5272.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.1084      0.115     -0.940      0.347      -0.334       0.118
sector[T.Consumer Discretionary]     0.0476      0.021      2.294      0.022       0.007       0.088
sector[T.Consumer Staples]           0.0823      0.016      5.245      0.000       0.052       0.113
sector[T.Energy]                     0.1584      0.018      8.894      0.000       0.123       0.193
sector[T.Financials]                 0.1452      0.018      8.039      0.000       0.110       0.181
sector[T.Health Care]                0.0091      0.018      0.496      0.620      -0.027       0.045
sector[T.Industrials]                0.0751      0.016      4.823      0.000       0.045       0.106
sector[T.Information Technology]     0.0688      0.022      3.194      0.001       0.027       0.111
sector[T.Materials]                  0.0992      0.015      6.413      0.000       0.069       0.129
sector[T.Real Estate]                0.1338      0.021      6.497      0.000       0.093       0.174
sector[T.Utilities]                  0.0992      0.016      6.334      0.000       0.068       0.130
ticks[T.AOT TB Equity]              -0.0088      0.017     -0.527      0.598      -0.041       0.024
ticks[T.AP TB Equity]                0.0591      0.013      4.658      0.000       0.034       0.084
ticks[T.BANPU TB Equity]             0.0309      0.012      2.482      0.013       0.006       0.055
ticks[T.BAY TB Equity]               0.0688      0.020      3.391      0.001       0.029       0.109
ticks[T.BBL TB Equity]               0.0751      0.022      3.346      0.001       0.031       0.119
ticks[T.BCH TB Equity]              -0.0133      0.019     -0.687      0.492      -0.051       0.025
ticks[T.BCP TB Equity]               0.0290      0.013      2.215      0.027       0.003       0.055
ticks[T.BDMS TB Equity]              0.0344      0.013      2.721      0.007       0.010       0.059
ticks[T.BEAUTY TB Equity]           -0.1165      0.037     -3.120      0.002      -0.190      -0.043
ticks[T.BEC TB Equity]               0.0086      0.032      0.268      0.789      -0.054       0.071
ticks[T.BEM TB Equity]              -0.0056      0.014     -0.410      0.682      -0.032       0.021
ticks[T.BGRIM TB Equity]            -0.0229      0.015     -1.518      0.129      -0.053       0.007
ticks[T.BH TB Equity]               -0.0120      0.013     -0.917      0.359      -0.038       0.014
ticks[T.BJC TB Equity]               0.0453      0.013      3.399      0.001       0.019       0.071
ticks[T.BTS TB Equity]               0.0292      0.012      2.479      0.013       0.006       0.052
ticks[T.CPALL TB Equity]            -0.0845      0.015     -5.599      0.000      -0.114      -0.055
ticks[T.CPF TB Equity]               0.0972      0.01

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     5.525
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           6.97e-31
Time:                        11:04:19   Log-Likelihood:                 456.05
No. Observations:                 672   AIC:                            -782.1
Df Residuals:                     607   BIC:                            -488.9
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -1.5631      0.695     -2.247      0.025      -2.929      -0.197
sector[T.Consumer Discretionary]     0.3057      0.086      3.575      0.000       0.138       0.474
sector[T.Consumer Staples]           0.0459      0.078      0.591      0.555      -0.107       0.199
sector[T.Energy]                     0.8209      0.103      8.000      0.000       0.619       1.022
sector[T.Financials]                 0.3828      0.119      3.209      0.001       0.149       0.617
sector[T.Health Care]               -0.0096      0.043     -0.224      0.823      -0.094       0.075
sector[T.Industrials]                0.1829      0.048      3.804      0.000       0.088       0.277
sector[T.Information Technology]     0.5834      0.160      3.636      0.000       0.268       0.898
sector[T.Materials]                  0.4564      0.059      7.683      0.000       0.340       0.573
sector[T.Real Estate]                0.7769      0.110      7.033      0.000       0.560       0.994
sector[T.Utilities]                  0.4759      0.105      4.530      0.000       0.270       0.682
ticks[T.AOT TB Equity]               0.2686      0.128      2.092      0.037       0.016       0.521
ticks[T.AP TB Equity]                0.0328      0.052      0.633      0.527      -0.069       0.135
ticks[T.AWC TB Equity]              -0.3207      0.184     -1.743      0.082      -0.682       0.041
ticks[T.BANPU TB Equity]             0.0556      0.056      0.989      0.323      -0.055       0.166
ticks[T.BAY TB Equity]               0.2093      0.131      1.593      0.112      -0.049       0.467
ticks[T.BBL TB Equity]              -0.2277      0.238     -0.957      0.339      -0.695       0.239
ticks[T.BCH TB Equity]              -0.2687      0.050     -5.387      0.000      -0.367      -0.171
ticks[T.BCP TB Equity]               0.1487      0.049      3.039      0.002       0.053       0.245
ticks[T.BDMS TB Equity]              0.0873      0.055      1.573      0.116      -0.022       0.196
ticks[T.BEAUTY TB Equity]            0.8617      0.512      1.683      0.093      -0.144       1.867
ticks[T.BEC TB Equity]               0.8651      0.281      3.082      0.002       0.314       1.416
ticks[T.BEM TB Equity]              -0.1219      0.047     -2.592      0.010      -0.214      -0.030
ticks[T.BGRIM TB Equity]            -0.3043      0.042     -7.183      0.000      -0.388      -0.221
ticks[T.BH TB Equity]                0.1718      0.087      1.986      0.048       0.002       0.342
ticks[T.BJC TB Equity]              -0.0197      0.106     -0.186      0.853      -0.228       0.188
ticks[T.BTS TB Equity]              -0.2551      0.123     -2.078      0.038      -0.496      -0.014
ticks[T.CPALL TB Equity]            -0.5932      0.12

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Return

ols_formula = 'RETURN ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RETURN   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     2.782
Date:                Thu, 04 Apr 2024   Prob (F-statistic):           5.13e-18
Time:                        11:04:29   Log-Likelihood:                 2080.6
No. Observations:                1682   AIC:                            -3939.
Df Residuals:                    1571   BIC:                            -3337.
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.7332      0.266     -2.759      0.006      -1.254      -0.212
sector[T.Consumer Discretionary]    -0.0649      0.015     -4.256      0.000      -0.095      -0.035
sector[T.Consumer Staples]          -0.0512      0.021     -2.396      0.017      -0.093      -0.009
sector[T.Energy]                     0.0401      0.046      0.877      0.380      -0.049       0.130
sector[T.Financials]                -0.0282      0.067     -0.422      0.673      -0.159       0.103
sector[T.Health Care]               -0.0409      0.013     -3.136      0.002      -0.066      -0.015
sector[T.Industrials]               -0.0684      0.040     -1.709      0.088      -0.147       0.010
sector[T.Information Technology]    -0.0605      0.013     -4.496      0.000      -0.087      -0.034
sector[T.Materials]                  0.0234      0.025      0.942      0.346      -0.025       0.072
sector[T.Real Estate]                0.0983      0.025      3.859      0.000       0.048       0.148
sector[T.Utilities]                 -0.0629      0.039     -1.615      0.107      -0.139       0.014
ticks[T.ACE TB Equity]               0.0794      0.045      1.757      0.079      -0.009       0.168
ticks[T.ADVANC TB Equity]           -0.2684      0.077     -3.478      0.001      -0.420      -0.117
ticks[T.AMATA TB Equity]            -0.0416      0.023     -1.812      0.070      -0.087       0.003
ticks[T.AOT TB Equity]              -0.2121      0.056     -3.779      0.000      -0.322      -0.102
ticks[T.AP TB Equity]                0.0245      0.017      1.477      0.140      -0.008       0.057
ticks[T.AWC TB Equity]               0.0321      0.037      0.880      0.379      -0.040       0.104
ticks[T.BAFS TB Equity]              0.0278      0.043      0.652      0.515      -0.056       0.111
ticks[T.BAM TB Equity]               0.0042      0.032      0.133      0.894      -0.058       0.066
ticks[T.BANPU TB Equity]            -0.0308      0.031     -0.992      0.321      -0.092       0.030
ticks[T.BAY TB Equity]               0.0475      0.053      0.894      0.371      -0.057       0.152
ticks[T.BBL TB Equity]               0.0137      0.064      0.212      0.832      -0.113       0.140
ticks[T.BCH TB Equity]              -0.0195      0.022     -0.872      0.384      -0.063       0.024
ticks[T.BCP TB Equity]               0.0390      0.018      2.116      0.034       0.003       0.075
ticks[T.BDMS TB Equity]             -0.1151      0.048     -2.387      0.017      -0.210      -0.021
ticks[T.BEAUTY TB Equity]            0.0394      0.088      0.448      0.654      -0.133       0.212
ticks[T.BEC TB Equity]              -0.0386      0.025     -1.558      0.119      -0.087       0.010
ticks[T.BEM TB Equity]              -0.0555      0.03

## vol

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     104.2
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        11:04:49   Log-Likelihood:                 949.25
No. Observations:                2464   AIC:                            -1777.
Df Residuals:                    2403   BIC:                            -1422.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            4.1468      0.252     16.467      0.000       3.653       4.641
sector[T.Consumer Discretionary]     0.1424      0.045      3.140      0.002       0.053       0.231
sector[T.Consumer Staples]          -0.0805      0.034     -2.348      0.019      -0.148      -0.013
sector[T.Energy]                     0.1121      0.039      2.881      0.004       0.036       0.188
sector[T.Financials]                 0.0288      0.039      0.731      0.465      -0.048       0.106
sector[T.Health Care]               -0.1773      0.040     -4.399      0.000      -0.256      -0.098
sector[T.Industrials]               -0.0902      0.034     -2.653      0.008      -0.157      -0.024
sector[T.Information Technology]     0.1457      0.047      3.096      0.002       0.053       0.238
sector[T.Materials]                  0.0979      0.034      2.899      0.004       0.032       0.164
sector[T.Real Estate]               -0.1638      0.045     -3.640      0.000      -0.252      -0.076
sector[T.Utilities]                 -0.0212      0.034     -0.620      0.535      -0.088       0.046
ticks[T.AOT TB Equity]               0.2520      0.036      6.920      0.000       0.181       0.323
ticks[T.AP TB Equity]               -0.0742      0.028     -2.677      0.007      -0.129      -0.020
ticks[T.BANPU TB Equity]            -0.0658      0.027     -2.422      0.016      -0.119      -0.013
ticks[T.BAY TB Equity]               0.3481      0.044      7.851      0.000       0.261       0.435
ticks[T.BBL TB Equity]              -0.2363      0.049     -4.820      0.000      -0.332      -0.140
ticks[T.BCH TB Equity]              -0.2702      0.042     -6.381      0.000      -0.353      -0.187
ticks[T.BCP TB Equity]              -0.2854      0.029     -9.968      0.000      -0.342      -0.229
ticks[T.BDMS TB Equity]              0.0367      0.028      1.329      0.184      -0.017       0.091
ticks[T.BEAUTY TB Equity]            0.4497      0.082      5.514      0.000       0.290       0.610
ticks[T.BEC TB Equity]               0.1177      0.070      1.687      0.092      -0.019       0.254
ticks[T.BEM TB Equity]              -0.2979      0.030    -10.052      0.000      -0.356      -0.240
ticks[T.BGRIM TB Equity]             0.0874      0.033      2.651      0.008       0.023       0.152
ticks[T.BH TB Equity]                0.0562      0.028      1.977      0.048       0.000       0.112
ticks[T.BJC TB Equity]               0.0889      0.029      3.052      0.002       0.032       0.146
ticks[T.BTS TB Equity]              -0.4554      0.026    -17.730      0.000      -0.506      -0.405
ticks[T.CPALL TB Equity]            -0.2793      0.033     -8.469      0.000      -0.344      -0.215
ticks[T.CPF TB Equity]               0.0267      0.03

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     65.61
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          1.51e-231
Time:                        11:05:11   Log-Likelihood:                 531.50
No. Observations:                 672   AIC:                            -933.0
Df Residuals:                     607   BIC:                            -639.8
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -2.2222      0.622     -3.575      0.000      -3.443      -1.001
sector[T.Consumer Discretionary]     1.1077      0.076     14.494      0.000       0.958       1.258
sector[T.Consumer Staples]          -0.1164      0.070     -1.674      0.095      -0.253       0.020
sector[T.Energy]                     0.3315      0.092      3.615      0.000       0.151       0.512
sector[T.Financials]                -0.4824      0.107     -4.524      0.000      -0.692      -0.273
sector[T.Health Care]                0.5890      0.038     15.389      0.000       0.514       0.664
sector[T.Industrials]                0.3530      0.043      8.214      0.000       0.269       0.437
sector[T.Information Technology]     1.1645      0.143      8.120      0.000       0.883       1.446
sector[T.Materials]                  0.2510      0.053      4.727      0.000       0.147       0.355
sector[T.Real Estate]                0.6261      0.099      6.342      0.000       0.432       0.820
sector[T.Utilities]                  0.6903      0.094      7.351      0.000       0.506       0.875
ticks[T.AOT TB Equity]               0.2970      0.115      2.589      0.010       0.072       0.522
ticks[T.AP TB Equity]                0.0358      0.046      0.774      0.439      -0.055       0.127
ticks[T.AWC TB Equity]              -0.7697      0.164     -4.682      0.000      -1.093      -0.447
ticks[T.BANPU TB Equity]            -0.0211      0.050     -0.419      0.675      -0.120       0.078
ticks[T.BAY TB Equity]              -1.0887      0.117     -9.271      0.000      -1.319      -0.858
ticks[T.BBL TB Equity]              -1.5653      0.213     -7.364      0.000      -1.983      -1.148
ticks[T.BCH TB Equity]               0.5262      0.045     11.804      0.000       0.439       0.614
ticks[T.BCP TB Equity]               0.1552      0.044      3.548      0.000       0.069       0.241
ticks[T.BDMS TB Equity]             -0.4686      0.050     -9.449      0.000      -0.566      -0.371
ticks[T.BEAUTY TB Equity]            2.9400      0.457      6.427      0.000       2.042       3.838
ticks[T.BEC TB Equity]               2.1038      0.251      8.387      0.000       1.611       2.596
ticks[T.BEM TB Equity]              -0.0007      0.042     -0.017      0.986      -0.083       0.082
ticks[T.BGRIM TB Equity]             0.2768      0.038      7.310      0.000       0.202       0.351
ticks[T.BH TB Equity]                0.5313      0.077      6.870      0.000       0.379       0.683
ticks[T.BJC TB Equity]              -0.4232      0.095     -4.470      0.000      -0.609      -0.237
ticks[T.BTS TB Equity]              -0.4600      0.110     -4.193      0.000      -0.675      -0.245
ticks[T.CPALL TB Equity]            -0.8295      0.11

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Vol

ols_formula = 'ln_vol ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ln_liq + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_vol   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     34.66
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:36:48   Log-Likelihood:                 451.17
No. Observations:                1682   AIC:                            -680.3
Df Residuals:                    1571   BIC:                            -77.87
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           12.0299      0.700     17.184      0.000      10.657      13.403
sector[T.Consumer Discretionary]     0.6578      0.040     16.382      0.000       0.579       0.737
sector[T.Consumer Staples]           0.7318      0.056     12.985      0.000       0.621       0.842
sector[T.Energy]                     1.8687      0.120     15.537      0.000       1.633       2.105
sector[T.Financials]                 2.6294      0.176     14.959      0.000       2.285       2.974
sector[T.Health Care]                0.0778      0.034      2.262      0.024       0.010       0.145
sector[T.Industrials]                1.3433      0.105     12.738      0.000       1.136       1.550
sector[T.Information Technology]     0.4657      0.035     13.131      0.000       0.396       0.535
sector[T.Materials]                  1.0969      0.066     16.744      0.000       0.968       1.225
sector[T.Real Estate]                0.9336      0.067     13.905      0.000       0.802       1.065
sector[T.Utilities]                  1.4628      0.103     14.261      0.000       1.262       1.664
ticks[T.ACE TB Equity]              -1.5300      0.119    -12.846      0.000      -1.764      -1.296
ticks[T.ADVANC TB Equity]            2.3108      0.203     11.362      0.000       1.912       2.710
ticks[T.AMATA TB Equity]            -0.4206      0.061     -6.947      0.000      -0.539      -0.302
ticks[T.AOT TB Equity]               0.9241      0.148      6.249      0.000       0.634       1.214
ticks[T.AP TB Equity]               -0.1402      0.044     -3.208      0.001      -0.226      -0.054
ticks[T.AWC TB Equity]               1.3834      0.096     14.375      0.000       1.195       1.572
ticks[T.BAFS TB Equity]             -1.2822      0.112    -11.412      0.000      -1.503      -1.062
ticks[T.BAM TB Equity]              -1.2076      0.083    -14.504      0.000      -1.371      -1.044
ticks[T.BANPU TB Equity]             1.0214      0.082     12.500      0.000       0.861       1.182
ticks[T.BAY TB Equity]               2.5367      0.140     18.117      0.000       2.262       2.811
ticks[T.BBL TB Equity]               2.2596      0.170     13.311      0.000       1.927       2.593
ticks[T.BCH TB Equity]              -0.4038      0.059     -6.855      0.000      -0.519      -0.288
ticks[T.BCP TB Equity]               0.4489      0.049      9.255      0.000       0.354       0.544
ticks[T.BDMS TB Equity]              1.4991      0.127     11.795      0.000       1.250       1.748
ticks[T.BEAUTY TB Equity]           -3.2802      0.232    -14.158      0.000      -3.735      -2.826
ticks[T.BEC TB Equity]              -0.4725      0.065     -7.244      0.000      -0.600      -0.345
ticks[T.BEM TB Equity]               0.0069      0.10

## liq

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_precovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     85.14
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:36:58   Log-Likelihood:                -1754.8
No. Observations:                2464   AIC:                             3630.
Df Residuals:                    2404   BIC:                             3978.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            3.6012      0.751      4.796      0.000       2.129       5.074
sector[T.Consumer Discretionary]     1.3409      0.133     10.074      0.000       1.080       1.602
sector[T.Consumer Staples]           0.5126      0.102      5.019      0.000       0.312       0.713
sector[T.Energy]                     1.0276      0.115      8.964      0.000       0.803       1.252
sector[T.Financials]                 0.2480      0.118      2.101      0.036       0.017       0.479
sector[T.Health Care]                0.5981      0.120      4.979      0.000       0.363       0.834
sector[T.Industrials]                0.7206      0.101      7.146      0.000       0.523       0.918
sector[T.Information Technology]     1.0147      0.139      7.278      0.000       0.741       1.288
sector[T.Materials]                  0.3836      0.101      3.803      0.000       0.186       0.581
sector[T.Real Estate]                1.1336      0.133      8.537      0.000       0.873       1.394
sector[T.Utilities]                  0.5761      0.102      5.661      0.000       0.377       0.776
ticks[T.AOT TB Equity]              -0.8069      0.108     -7.481      0.000      -1.018      -0.595
ticks[T.AP TB Equity]                0.4342      0.083      5.259      0.000       0.272       0.596
ticks[T.BANPU TB Equity]             1.1343      0.078     14.544      0.000       0.981       1.287
ticks[T.BAY TB Equity]              -3.8524      0.107    -35.977      0.000      -4.062      -3.642
ticks[T.BBL TB Equity]              -0.3596      0.147     -2.451      0.014      -0.647      -0.072
ticks[T.BCH TB Equity]               1.0854      0.125      8.688      0.000       0.840       1.330
ticks[T.BCP TB Equity]               0.1859      0.086      2.169      0.030       0.018       0.354
ticks[T.BDMS TB Equity]             -0.4355      0.082     -5.292      0.000      -0.597      -0.274
ticks[T.BEAUTY TB Equity]            1.7162      0.242      7.097      0.000       1.242       2.190
ticks[T.BEC TB Equity]               1.3592      0.207      6.561      0.000       0.953       1.765
ticks[T.BEM TB Equity]               0.2538      0.089      2.863      0.004       0.080       0.428
ticks[T.BGRIM TB Equity]             0.5033      0.098      5.124      0.000       0.311       0.696
ticks[T.BH TB Equity]               -0.0517      0.085     -0.607      0.544      -0.219       0.115
ticks[T.BJC TB Equity]              -0.2716      0.087     -3.120      0.002      -0.442      -0.101
ticks[T.BTS TB Equity]              -0.0745      0.077     -0.968      0.333      -0.225       0.076
ticks[T.CPALL TB Equity]            -0.4899      0.098     -4.985      0.000      -0.683      -0.297
ticks[T.CPF TB Equity]               0.3494      0.11

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_G + (esg_G*stringency_index) + stringency_index + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_covid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     31.58
Date:                Thu, 04 Apr 2024   Prob (F-statistic):          9.89e-153
Time:                        14:37:09   Log-Likelihood:                -420.53
No. Observations:                 672   AIC:                             969.1
Df Residuals:                     608   BIC:                             1258.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            9.9883      2.529      3.950      0.000       5.022      14.955
sector[T.Consumer Discretionary]     1.1449      0.311      3.676      0.000       0.533       1.757
sector[T.Consumer Staples]           1.7749      0.277      6.402      0.000       1.230       2.319
sector[T.Energy]                     1.0564      0.375      2.814      0.005       0.319       1.794
sector[T.Financials]                 2.1561      0.431      5.008      0.000       1.311       3.002
sector[T.Health Care]                0.4695      0.157      2.999      0.003       0.162       0.777
sector[T.Industrials]                0.7502      0.174      4.301      0.000       0.408       1.093
sector[T.Information Technology]     0.2650      0.591      0.448      0.654      -0.895       1.425
sector[T.Materials]                  0.5881      0.217      2.704      0.007       0.161       1.015
sector[T.Real Estate]                0.6823      0.406      1.681      0.093      -0.115       1.479
sector[T.Utilities]                  0.0372      0.387      0.096      0.923      -0.723       0.797
ticks[T.AOT TB Equity]              -2.2384      0.464     -4.824      0.000      -3.150      -1.327
ticks[T.AP TB Equity]                1.2109      0.184      6.567      0.000       0.849       1.573
ticks[T.AWC TB Equity]               1.2025      0.676      1.780      0.076      -0.124       2.529
ticks[T.BANPU TB Equity]             1.7042      0.195      8.738      0.000       1.321       2.087
ticks[T.BAY TB Equity]              -2.4982      0.473     -5.281      0.000      -3.427      -1.569
ticks[T.BBL TB Equity]               3.0051      0.867      3.465      0.001       1.302       4.708
ticks[T.BCH TB Equity]               1.0950      0.178      6.144      0.000       0.745       1.445
ticks[T.BCP TB Equity]              -0.4546      0.179     -2.536      0.011      -0.807      -0.103
ticks[T.BDMS TB Equity]             -0.0573      0.204     -0.281      0.779      -0.459       0.344
ticks[T.BEAUTY TB Equity]           -3.4028      1.880     -1.810      0.071      -7.095       0.289
ticks[T.BEC TB Equity]              -0.2103      1.034     -0.204      0.839      -2.240       1.819
ticks[T.BEM TB Equity]               0.3349      0.173      1.940      0.053      -0.004       0.674
ticks[T.BGRIM TB Equity]             0.2921      0.156      1.877      0.061      -0.013       0.598
ticks[T.BH TB Equity]               -0.5682      0.318     -1.788      0.074      -1.192       0.056
ticks[T.BJC TB Equity]               1.3764      0.386      3.565      0.000       0.618       2.135
ticks[T.BTS TB Equity]               1.5181      0.448      3.390      0.001       0.639       2.398
ticks[T.CPALL TB Equity]             0.8544      0.47

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Liq

ols_formula = 'ln_liq ~ esg_G + ln_total_asset + RETURN_ON_ASSET + ln_debt_to_e + ln_PTB + SALES_GROWTH + sector + ticks '
OLS = ols(formula = ols_formula, data = df_postcovid)
model = OLS.fit()
model_results = model.summary()
model_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ln_liq   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     62.28
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:37:25   Log-Likelihood:                -1095.7
No. Observations:                1682   AIC:                             2411.
Df Residuals:                    1572   BIC:                             3008.
Df Model:                         109                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            9.6096      1.739      5.527      0.000       6.199      13.020
sector[T.Consumer Discretionary]     0.1976      0.101      1.965      0.050       0.000       0.395
sector[T.Consumer Staples]           0.3386      0.141      2.400      0.017       0.062       0.615
sector[T.Energy]                     2.1357      0.297      7.197      0.000       1.554       2.718
sector[T.Financials]                 1.7718      0.439      4.041      0.000       0.912       2.632
sector[T.Health Care]                0.0674      0.086      0.782      0.434      -0.102       0.236
sector[T.Industrials]                0.6793      0.264      2.574      0.010       0.162       1.197
sector[T.Information Technology]     0.7547      0.087      8.687      0.000       0.584       0.925
sector[T.Materials]                  0.2190      0.164      1.334      0.182      -0.103       0.541
sector[T.Real Estate]                1.0376      0.166      6.238      0.000       0.711       1.364
sector[T.Utilities]                  0.6999      0.257      2.727      0.006       0.197       1.203
ticks[T.ACE TB Equity]              -0.8496      0.298     -2.852      0.004      -1.434      -0.265
ticks[T.ADVANC TB Equity]           -0.3479      0.510     -0.682      0.495      -1.348       0.652
ticks[T.AMATA TB Equity]             1.1151      0.149      7.474      0.000       0.822       1.408
ticks[T.AOT TB Equity]              -1.6431      0.369     -4.459      0.000      -2.366      -0.920
ticks[T.AP TB Equity]                1.3057      0.105     12.491      0.000       1.101       1.511
ticks[T.AWC TB Equity]               0.4849      0.241      2.012      0.044       0.012       0.958
ticks[T.BAFS TB Equity]             -1.6601      0.279     -5.958      0.000      -2.207      -1.114
ticks[T.BAM TB Equity]               0.3950      0.209      1.894      0.058      -0.014       0.804
ticks[T.BANPU TB Equity]             2.0307      0.198     10.236      0.000       1.642       2.420
ticks[T.BAY TB Equity]              -1.7142      0.348     -4.920      0.000      -2.398      -1.031
ticks[T.BBL TB Equity]               2.6854      0.420      6.390      0.000       1.861       3.510
ticks[T.BCH TB Equity]               0.7264      0.147      4.956      0.000       0.439       1.014
ticks[T.BCP TB Equity]               0.3515      0.121      2.898      0.004       0.114       0.589
ticks[T.BDMS TB Equity]             -0.0144      0.319     -0.045      0.964      -0.640       0.611
ticks[T.BEAUTY TB Equity]           -1.2840      0.580     -2.213      0.027      -2.422      -0.146
ticks[T.BEC TB Equity]               0.1504      0.164      0.920      0.358      -0.170       0.471
ticks[T.BEM TB Equity]              -0.0473      0.25